## Tuning di merging lists
Usa classe del prof per fsare recommender, è di hybridV4


In [31]:
import sys
sys.path.insert(0, './Lab/')
sys.path.insert(0, './New_Hybrids/')

from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from UserIcmKNNCFRecommender import UserIcmKNNCFRecommender
from ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
import matplotlib.pyplot as plt
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from WeightedHybridV3V4forBayesianSearch import WeightedHybridV3V4forBayesianSearch
from FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg
# from WeightedHybridV4forBayesianSearch import WeightedHybridV4forBayesianSearch
%matplotlib inline 

import numpy as np

In [32]:
from optimize_weights_notcross import optimize_weights

In [33]:
from optimize_weights_1at1 import optimize_weights_1at1

In [34]:
sys.path.insert(0, './datasets/')
from load_tables import load_ICM_csr, load_URM_csr
ICM_all = load_ICM_csr()
URM_all = load_URM_csr()
URM_train = URM_all

Deallocating Cython objects


In [35]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Notebooks_utils.data_splitter import train_test_holdout
from Base.Evaluation.Evaluator import EvaluatorHoldout

# URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.95)#diminuisco size del test 

# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

## TUNING

In [36]:
userknn = {}
userknn["topK"] = 121
userknn["shrink"] = 2

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 254
cy["topK"] = 881
cy["positive_threshold_BPR"] = 0.9765
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565

hybridv2={}
#bayesianSearchhybridV2
#recommenders usati: [alpha,beta,userknn,uicm,cy,itemknn,iicm]
hybridv2["w0"]=0.001
hybridv2["w1"]=0.5593506236737317
hybridv2["w2"]=0.23267769028384777
hybridv2["w3"]=0.10077656126622785
hybridv2["w4"]=0.0001
hybridv2["w5"]=0.0001
hybridv2["w6"]=0.8397993296794579

itemcbf={}
itemcbf["topK"]=107
itemcbf["shrink"]=18.384308506219117
itemcbf["normalize"] = True
itemcbf["feature_weighting"] = 'BM25'
itemcbf["similarity"] = 'jaccard'

cfw_icm =  {}
cfw_icm["topK"] = 702
cfw_icm["add_zeros_quota"] = 0.975154924862614
cfw_icm['normalize_similarity'] = False

cfw_knn =  {}
cfw_knn["topK"] = 128
cfw_knn["add_zeros_quota"] = 0.4586460222330323
cfw_knn['normalize_similarity'] = False


# 6 recomm
fits = [alpha,beta,userknn,uicm,cy,cfw_icm,cfw_knn]


recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    #UserKNNCFRecommender,
    IALSRecommender,
    ItemIcmKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    #ItemIcmKNNCFRecommender,
    #IALSRecommender,
    CFW_D_Similarity_Linalg,
    CFW_D_Similarity_Linalg,
    ItemKNNCBFRecommender
    ]



only_URM = {
        "URM_train" : URM_train
    }

cython = {
        "URM_train" : URM_train,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_train,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_train,
        "ICM_train" : ICM_all
    }

cfw  =  {
        "URM_train" : URM_train,
        "ICM" : ICM_all,
        "S_matrix_target": None
    }

inits = [
        only_URM,only_URM,only_URM,also_ICM,cython,cfw,cfw,CBF
    ]

In [ ]:
ww_std = optimize_weights(URM_train,recs, inits, fits,3,weights = [1, 1.0865713662719998, 1.0147173261192,1.8,0.036,1,1],index =5, normalize_scores = True)
#[1, 1.0865713662719998, 1.0147173261192, 1.8, 0.036, 1.5,0.45]


In [7]:
ww_not_std = optimize_weights(URM_train,recs, inits, fits,3, normalize_scores = False)
#[1, 0.8, 0.00138, 1.1976, 0.099, 0.6,0.552]


optimize 1
WeightedHybridScoreRecommender: URM Detected 140 (1.76 %) cold users.
WeightedHybridScoreRecommender: URM Detected 3197 (12.31 %) cold items.
TopPopRecommender: URM Detected 140 (1.76 %) cold users.
TopPopRecommender: URM Detected 3197 (12.31 %) cold items.
P3alphaRecommender: URM Detected 140 (1.76 %) cold users.
P3alphaRecommender: URM Detected 3197 (12.31 %) cold items.
RP3betaRecommender: URM Detected 140 (1.76 %) cold users.
RP3betaRecommender: URM Detected 3197 (12.31 %) cold items.
--------FITTING START-------
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: RP3betaRecommender-------
------FITTING END------
trials [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
weights : [1, 0.0001]
EvaluatorHoldout: Processed 5384 ( 100.00% ) in 15.29 sec. Users per second: 352
MAP 0.051716041883568716 WITH WEIGHTS [1, 0.0001]
weights : [1, 0.001]
EvaluatorHoldout: Processed 5384 ( 100.00% ) in 14.88 sec. Users per second: 362
MAP 0.05171776165683382 WITH WE

WeightedHybridScoreRecommender: URM Detected 139 (1.75 %) cold users.
WeightedHybridScoreRecommender: URM Detected 3181 (12.25 %) cold items.
TopPopRecommender: URM Detected 139 (1.75 %) cold users.
TopPopRecommender: URM Detected 3181 (12.25 %) cold items.
P3alphaRecommender: URM Detected 139 (1.75 %) cold users.
P3alphaRecommender: URM Detected 3181 (12.25 %) cold items.
RP3betaRecommender: URM Detected 139 (1.75 %) cold users.
RP3betaRecommender: URM Detected 3181 (12.25 %) cold items.
UserKNNCFRecommender: URM Detected 139 (1.75 %) cold users.
UserKNNCFRecommender: URM Detected 3181 (12.25 %) cold items.
--------FITTING START-------
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: RP3betaRecommender-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 9886.87 column/sec, elapsed time 0.01 min
------FITTING END------
trials [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
weights : [1, 0.8, 0.0001]
EvaluatorHoldou

EvaluatorHoldout: Processed 5442 ( 100.00% ) in 45.13 sec. Users per second: 121
MAP 0.0490320549133124 WITH WEIGHTS [1, 0.8, 0.001485]
weights : [1, 0.8, 0.0014925000000000001]
EvaluatorHoldout: Processed 2000 ( 36.75% ) in 35.12 sec. Users per second: 57
EvaluatorHoldout: Processed 4000 ( 73.50% ) in 1.12 min. Users per second: 59
EvaluatorHoldout: Processed 5442 ( 100.00% ) in 1.42 min. Users per second: 64
MAP 0.0490320549133124 WITH WEIGHTS [1, 0.8, 0.0014925000000000001]
weights : [1, 0.8, 0.001497]
EvaluatorHoldout: Processed 4000 ( 73.50% ) in 30.32 sec. Users per second: 132
EvaluatorHoldout: Processed 5442 ( 100.00% ) in 39.33 sec. Users per second: 138
MAP 0.0490320549133124 WITH WEIGHTS [1, 0.8, 0.001497]
weights : [1, 0.8, 0.0015]
EvaluatorHoldout: Processed 5000 ( 91.88% ) in 32.38 sec. Users per second: 154
EvaluatorHoldout: Processed 5442 ( 100.00% ) in 35.48 sec. Users per second: 153
MAP 0.0490320549133124 WITH WEIGHTS [1, 0.8, 0.0015]
weights : [1, 0.8, 0.00153000000

MAP 0.05417672985867901 WITH WEIGHTS [1, 0.8, 0.00138, 0.9]
weights : [1, 0.8, 0.00138, 0.95]
EvaluatorHoldout: Processed 4000 ( 74.57% ) in 30.56 sec. Users per second: 131
EvaluatorHoldout: Processed 5364 ( 100.00% ) in 41.13 sec. Users per second: 130
MAP 0.05397645610913726 WITH WEIGHTS [1, 0.8, 0.00138, 0.95]
weights : [1, 0.8, 0.00138, 0.98]
EvaluatorHoldout: Processed 5000 ( 93.21% ) in 37.41 sec. Users per second: 134
EvaluatorHoldout: Processed 5364 ( 100.00% ) in 40.07 sec. Users per second: 134
MAP 0.05413016365507166 WITH WEIGHTS [1, 0.8, 0.00138, 0.98]
weights : [1, 0.8, 0.00138, 1]
EvaluatorHoldout: Processed 4000 ( 74.57% ) in 32.19 sec. Users per second: 124
EvaluatorHoldout: Processed 5364 ( 100.00% ) in 41.74 sec. Users per second: 129
MAP 0.05414190999541738 WITH WEIGHTS [1, 0.8, 0.00138, 1]
weights : [1, 0.8, 0.00138, 1.2]
EvaluatorHoldout: Processed 4000 ( 74.57% ) in 30.52 sec. Users per second: 131
EvaluatorHoldout: Processed 5364 ( 100.00% ) in 41.55 sec. Users 

UserKNNCFRecommender: URM Detected 141 (1.77 %) cold users.
UserKNNCFRecommender: URM Detected 60 (0.30 %) cold items.
--------FITTING START-------
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: RP3betaRecommender-------
RP3betaRecommender: Processed 14400 ( 55.44% ) in 1.01 minutes. Rows per second: 238
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 3103.53 column/sec, elapsed time 0.04 min
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 2722.52 column/sec, elapsed time 0.05 min
Similarity column 7947 ( 100 % ), 648.31 column/sec, elapsed time 0.20 min
--------FITTING IN PROGRESS: SLIM_BPR_Recommender-------
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 1.17 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 2.73 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 3.94 sec
SLIM_BPR_Recommender: Epo

...

Similarity column 25975 ( 100 % ), 2682.86 column/sec, elapsed time 0.16 min
ItemKNNCFRecommender: URM Detected 140 (1.76 %) cold users.
ItemKNNCFRecommender: URM Detected 3251 (12.52 %) cold items.
Similarity column 25975 ( 100 % ), 3498.90 column/sec, elapsed time 0.12 min
WeightedHybridScoreRecommender: URM Detected 140 (1.76 %) cold users.
WeightedHybridScoreRecommender: URM Detected 3251 (12.52 %) cold items.
TopPopRecommender: URM Detected 140 (1.76 %) cold users.
TopPopRecommender: URM Detected 3251 (12.52 %) cold items.
P3alphaRecommender: URM Detected 140 (1.76 %) cold users.
P3alphaRecommender: URM Detected 3251 (12.52 %) cold items.
RP3betaRecommender: URM Detected 140 (1.76 %) cold users.
RP3betaRecommender: URM Detected 3251 (12.52 %) cold items.
UserKNNCFRecommender: URM Detected 140 (1.76 %) cold users.
UserKNNCFRecommender: URM Detected 3251 (12.52 %) cold items.
UserKNNCFRecommender: URM Detected 140 (1.76 %) cold users.
UserKNNCFRecommender: URM Detected 3251 (12.52 %

SLIM_BPR_Recommender: Epoch 103 of 254. Elapsed time 4.21 sec
SLIM_BPR_Recommender: Epoch 104 of 254. Elapsed time 4.24 sec
SLIM_BPR_Recommender: Epoch 105 of 254. Elapsed time 4.26 sec
SLIM_BPR_Recommender: Epoch 106 of 254. Elapsed time 4.29 sec
SLIM_BPR_Recommender: Epoch 107 of 254. Elapsed time 4.31 sec
SLIM_BPR_Recommender: Epoch 108 of 254. Elapsed time 4.34 sec
SLIM_BPR_Recommender: Epoch 109 of 254. Elapsed time 4.36 sec
SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 4.38 sec
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 4.40 sec
SLIM_BPR_Recommender: Epoch 112 of 254. Elapsed time 4.44 sec
SLIM_BPR_Recommender: Epoch 113 of 254. Elapsed time 4.47 sec
SLIM_BPR_Recommender: Epoch 114 of 254. Elapsed time 4.50 sec
SLIM_BPR_Recommender: Epoch 115 of 254. Elapsed time 4.52 sec
SLIM_BPR_Recommender: Epoch 116 of 254. Elapsed time 4.55 sec
SLIM_BPR_Recommender: Epoch 117 of 254. Elapsed time 4.57 sec
SLIM_BPR_Recommender: Epoch 118 of 254. Elapsed time 4.59 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 236 of 254. Elapsed time 7.41 sec
SLIM_BPR_Recommender: Epoch 237 of 254. Elapsed time 7.43 sec
SLIM_BPR_Recommender: Epoch 238 of 254. Elapsed time 7.46 sec
SLIM_BPR_Recommender: Epoch 239 of 254. Elapsed time 7.48 sec
SLIM_BPR_Recommender: Epoch 240 of 254. Elapsed time 7.50 sec
SLIM_BPR_Recommender: Epoch 241 of 254. Elapsed time 7.52 sec
SLIM_BPR_Recommender: Epoch 242 of 254. Elapsed time 7.54 sec
SLIM_BPR_Recommender: Epoch 243 of 254. Elapsed time 7.56 sec
SLIM_BPR_Recommender: Epoch 244 of 254. Elapsed time 7.57 sec
SLIM_BPR_Recommender: Epoch 245 of 254. Elapsed time 7.59 sec
SLIM_BPR_Recommender: Epoch 246 of 254. Elapsed time 7.62 sec
SLIM_BPR_Recommender: Epoch 247 of 254. Elapsed time 7.64 sec
SLIM_BPR_Recommender: Epoch 248 of 254. Elapsed time 7.66 sec
SLIM_BPR_Recommender: Epoch 249 of 254. Elapsed time 7.68 sec
SLIM_BPR_Recommender: Epoch 250 of 254. Elapsed time 7.70 sec
SLIM_BPR_Recommender: Epoch 251 of 254. Elapsed time 7.72 sec
SLIM_BPR

EvaluatorHoldout: Processed 5403 ( 100.00% ) in 41.89 sec. Users per second: 129
MAP 0.05636168506928132 WITH WEIGHTS [1, 0.8, 0.00138, 1.1976, 0.099, 0.6, 0.098]
weights : [1, 0.8, 0.00138, 1.1976, 0.099, 0.6, 0.1]
EvaluatorHoldout: Processed 4000 ( 74.03% ) in 38.09 sec. Users per second: 105
EvaluatorHoldout: Processed 5403 ( 100.00% ) in 49.07 sec. Users per second: 110
MAP 0.05639853996177197 WITH WEIGHTS [1, 0.8, 0.00138, 1.1976, 0.099, 0.6, 0.1]
weights : [1, 0.8, 0.00138, 1.1976, 0.099, 0.6, 0.12000000000000001]
EvaluatorHoldout: Processed 4000 ( 74.03% ) in 32.55 sec. Users per second: 123
EvaluatorHoldout: Processed 5403 ( 100.00% ) in 43.24 sec. Users per second: 125
MAP 0.05638910155990057 WITH WEIGHTS [1, 0.8, 0.00138, 1.1976, 0.099, 0.6, 0.12000000000000001]
weights : [1, 0.8, 0.00138, 1.1976, 0.099, 0.6, 0.15000000000000002]
EvaluatorHoldout: Processed 4000 ( 74.03% ) in 35.55 sec. Users per second: 113
EvaluatorHoldout: Processed 5403 ( 100.00% ) in 46.89 sec. Users per

Deallocating Cython objects
Deallocating Cython objects


In [8]:
ww_std = [1, 1.08657, 0.5958, 1, 0.036, 0.7875,0.44]
ww_not_std =  [1.212, 0.8, 0.00138, 0.8982, 0.1, 0.0006,0.4987505]

In [10]:
ww_std2 = optimize_weights_1at1(URM_train,recs, inits, fits,3,weights = ww_std, normalize_scores = True)


ItemKNNCFRecommender: URM Detected 102 (1.28 %) cold users.
ItemKNNCFRecommender: URM Detected 2693 (10.37 %) cold items.
Similarity column 25975 ( 100 % ), 3130.29 column/sec, elapsed time 0.14 min
ItemKNNCFRecommender: URM Detected 102 (1.28 %) cold users.
ItemKNNCFRecommender: URM Detected 2693 (10.37 %) cold items.
Similarity column 25975 ( 100 % ), 3055.75 column/sec, elapsed time 0.14 min
WeightedHybridScoreRecommender: URM Detected 102 (1.28 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2693 (10.37 %) cold items.
TopPopRecommender: URM Detected 102 (1.28 %) cold users.
TopPopRecommender: URM Detected 2693 (10.37 %) cold items.
P3alphaRecommender: URM Detected 102 (1.28 %) cold users.
P3alphaRecommender: URM Detected 2693 (10.37 %) cold items.
RP3betaRecommender: URM Detected 102 (1.28 %) cold users.
RP3betaRecommender: URM Detected 2693 (10.37 %) cold items.
UserKNNCFRecommender: URM Detected 102 (1.28 %) cold users.
UserKNNCFRecommender: URM Detected 2693 (10.37 %

SLIM_BPR_Recommender: Epoch 100 of 254. Elapsed time 5.00 sec
SLIM_BPR_Recommender: Epoch 101 of 254. Elapsed time 5.04 sec
SLIM_BPR_Recommender: Epoch 102 of 254. Elapsed time 5.08 sec
SLIM_BPR_Recommender: Epoch 103 of 254. Elapsed time 5.11 sec
SLIM_BPR_Recommender: Epoch 104 of 254. Elapsed time 5.15 sec
SLIM_BPR_Recommender: Epoch 105 of 254. Elapsed time 5.18 sec
SLIM_BPR_Recommender: Epoch 106 of 254. Elapsed time 5.21 sec
SLIM_BPR_Recommender: Epoch 107 of 254. Elapsed time 5.25 sec
SLIM_BPR_Recommender: Epoch 108 of 254. Elapsed time 5.28 sec
SLIM_BPR_Recommender: Epoch 109 of 254. Elapsed time 5.31 sec
SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 5.34 sec
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 5.39 sec
SLIM_BPR_Recommender: Epoch 112 of 254. Elapsed time 5.43 sec
SLIM_BPR_Recommender: Epoch 113 of 254. Elapsed time 5.46 sec
SLIM_BPR_Recommender: Epoch 114 of 254. Elapsed time 5.50 sec
SLIM_BPR_Recommender: Epoch 115 of 254. Elapsed time 5.54 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 233 of 254. Elapsed time 9.27 sec
SLIM_BPR_Recommender: Epoch 234 of 254. Elapsed time 9.31 sec
SLIM_BPR_Recommender: Epoch 235 of 254. Elapsed time 9.35 sec
SLIM_BPR_Recommender: Epoch 236 of 254. Elapsed time 9.39 sec
SLIM_BPR_Recommender: Epoch 237 of 254. Elapsed time 9.43 sec
SLIM_BPR_Recommender: Epoch 238 of 254. Elapsed time 9.48 sec
SLIM_BPR_Recommender: Epoch 239 of 254. Elapsed time 9.52 sec
SLIM_BPR_Recommender: Epoch 240 of 254. Elapsed time 9.56 sec
SLIM_BPR_Recommender: Epoch 241 of 254. Elapsed time 9.61 sec
SLIM_BPR_Recommender: Epoch 242 of 254. Elapsed time 9.65 sec
SLIM_BPR_Recommender: Epoch 243 of 254. Elapsed time 9.78 sec
SLIM_BPR_Recommender: Epoch 244 of 254. Elapsed time 9.83 sec
SLIM_BPR_Recommender: Epoch 245 of 254. Elapsed time 9.87 sec
SLIM_BPR_Recommender: Epoch 246 of 254. Elapsed time 9.91 sec
SLIM_BPR_Recommender: Epoch 247 of 254. Elapsed time 9.96 sec
SLIM_BPR_Recommender: Epoch 248 of 254. Elapsed time 10.00 sec
SLIM_BP

EvaluatorHoldout: Processed 5508 ( 100.00% ) in 47.96 sec. Users per second: 115
MAP 0.061654666038797754 WITH WEIGHTS [1, 5.0, 0.5958, 1, 0.036, 0.7875, 0.44]
weights : [1, 6.0, 0.5958, 1, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 35.67 sec. Users per second: 112
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 47.36 sec. Users per second: 116
MAP 0.061454978482019224 WITH WEIGHTS [1, 6.0, 0.5958, 1, 0.036, 0.7875, 0.44]
weights : [1, 7.0, 0.5958, 1, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 31.10 sec. Users per second: 129
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 42.64 sec. Users per second: 129
MAP 0.061422987261891465 WITH WEIGHTS [1, 7.0, 0.5958, 1, 0.036, 0.7875, 0.44]
weights : [1, 8.0, 0.5958, 1, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 31.47 sec. Users per second: 127
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 43.32 sec. Users per second: 127
MAP 0.060844064734947474 WITH WEIGHTS [1, 8.0,

EvaluatorHoldout: Processed 4000 ( 72.62% ) in 34.79 sec. Users per second: 115
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 46.20 sec. Users per second: 119
MAP 0.050655632366115054 WITH WEIGHTS [1, 2.6, 0.5958, 80, 0.036, 0.7875, 0.44]
BEST MAP 0.062211832406716544 WITH T 1
trials [0.6 0.7 0.8 0.9 1.  1.8 2.6 3.4 4.2]
weights : [1, 2.6, 0.5958, 0.6, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 30.58 sec. Users per second: 131
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 41.84 sec. Users per second: 132
MAP 0.061484843623091666 WITH WEIGHTS [1, 2.6, 0.5958, 0.6, 0.036, 0.7875, 0.44]
weights : [1, 2.6, 0.5958, 0.7, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 31.37 sec. Users per second: 128
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 42.84 sec. Users per second: 129
MAP 0.06159829286652926 WITH WEIGHTS [1, 2.6, 0.5958, 0.7, 0.036, 0.7875, 0.44]
weights : [1, 2.6, 0.5958, 0.8, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 

...

EvaluatorHoldout: Processed 5508 ( 100.00% ) in 51.92 sec. Users per second: 106
MAP 0.06157893727335458 WITH WEIGHTS [0.9400000000000001, 2.6, 0.0001, 1.0, 0.036, 0.7875, 0.44]
weights : [0.9400000000000001, 2.6, 0.0005, 1.0, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 33.98 sec. Users per second: 118
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 45.84 sec. Users per second: 120
MAP 0.061578613069599 WITH WEIGHTS [0.9400000000000001, 2.6, 0.0005, 1.0, 0.036, 0.7875, 0.44]
weights : [0.9400000000000001, 2.6, 0.001, 1.0, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 31.48 sec. Users per second: 127
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 43.13 sec. Users per second: 128
MAP 0.061578613069599 WITH WEIGHTS [0.9400000000000001, 2.6, 0.001, 1.0, 0.036, 0.7875, 0.44]
weights : [0.9400000000000001, 2.6, 0.005, 1.0, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 31.48 sec. Users per second: 127
EvaluatorHoldout: Proce

EvaluatorHoldout: Processed 4000 ( 72.62% ) in 34.49 sec. Users per second: 116
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 48.16 sec. Users per second: 114
MAP 0.062133776578761304 WITH WEIGHTS [0.9400000000000001, 2.6, 0.45999999999999996, 1.0, 0.036, 0.7875, 0.44]
weights : [0.9400000000000001, 2.6, 0.6, 1.0, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 36.51 sec. Users per second: 110
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 48.33 sec. Users per second: 114
MAP 0.062270921857540344 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 1.0, 0.036, 0.7875, 0.44]
weights : [0.9400000000000001, 2.6, 0.62, 1.0, 0.036, 0.7875, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 30.28 sec. Users per second: 132
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 41.76 sec. Users per second: 132
MAP 0.062177201556205326 WITH WEIGHTS [0.9400000000000001, 2.6, 0.62, 1.0, 0.036, 0.7875, 0.44]
weights : [0.9400000000000001, 2.6, 0.64, 1.0, 0.036, 0.7875, 0.44]
EvaluatorHol

MAP 0.060934651295360326 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.005, 0.44]
weights : [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.01, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 30.68 sec. Users per second: 130
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 42.34 sec. Users per second: 130
MAP 0.06095037157524183 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.01, 0.44]
weights : [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.05, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 31.17 sec. Users per second: 128
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 43.18 sec. Users per second: 128
MAP 0.06106277862082182 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.05, 0.44]
weights : [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.1, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 30.28 sec. Users per second: 132
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 41.99 sec. Users per second: 131
MAP 0.061080223098612954 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 

EvaluatorHoldout: Processed 5508 ( 100.00% ) in 50.29 sec. Users per second: 110
MAP 0.06231547245670089 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.6599999999999999, 0.44]
weights : [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.6799999999999999, 0.44]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 33.16 sec. Users per second: 121
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 44.66 sec. Users per second: 123
MAP 0.0621986174156447 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.6799999999999999, 0.44]
BEST MAP 0.06238595435316319 WITH T 0.6
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
BEST T:0.6 -> BEST MAP:0.06238595435316319
optimize 6
trials [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 80]
weigh

MAP 0.0623281118574543 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.6, 0.52]
weights : [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.6, 0.54]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 31.39 sec. Users per second: 127
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 44.12 sec. Users per second: 125
MAP 0.06234586861722268 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.6, 0.54]
weights : [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.6, 0.5599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 32.69 sec. Users per second: 122
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 46.41 sec. Users per second: 119
MAP 0.06236569213304881 WITH WEIGHTS [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.6, 0.5599999999999999]
weights : [0.9400000000000001, 2.6, 0.6, 1.0, 0, 0.6, 0.58]
EvaluatorHoldout: Processed 4000 ( 72.62% ) in 36.69 sec. Users per second: 109
EvaluatorHoldout: Processed 5508 ( 100.00% ) in 50.47 sec. Users per second: 109
MAP 0.06250100943440924 WITH WEIGHTS [0.940000000

In [9]:
ww_not_std2 = optimize_weights_1at1(URM_train,recs, inits, fits,3,weights = ww_not_std, normalize_scores = False)


ItemKNNCFRecommender: URM Detected 99 (1.25 %) cold users.
ItemKNNCFRecommender: URM Detected 2725 (10.49 %) cold items.
Similarity column 25975 ( 100 % ), 2212.56 column/sec, elapsed time 0.20 min
ItemKNNCFRecommender: URM Detected 99 (1.25 %) cold users.
ItemKNNCFRecommender: URM Detected 2725 (10.49 %) cold items.
Similarity column 25975 ( 100 % ), 2154.55 column/sec, elapsed time 0.20 min
WeightedHybridScoreRecommender: URM Detected 99 (1.25 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2725 (10.49 %) cold items.
TopPopRecommender: URM Detected 99 (1.25 %) cold users.
TopPopRecommender: URM Detected 2725 (10.49 %) cold items.
P3alphaRecommender: URM Detected 99 (1.25 %) cold users.
P3alphaRecommender: URM Detected 2725 (10.49 %) cold items.
RP3betaRecommender: URM Detected 99 (1.25 %) cold users.
RP3betaRecommender: URM Detected 2725 (10.49 %) cold items.
UserKNNCFRecommender: URM Detected 99 (1.25 %) cold users.
UserKNNCFRecommender: URM Detected 2725 (10.49 %) cold 

SLIM_BPR_Recommender: Epoch 100 of 254. Elapsed time 6.57 sec
SLIM_BPR_Recommender: Epoch 101 of 254. Elapsed time 6.63 sec
SLIM_BPR_Recommender: Epoch 102 of 254. Elapsed time 6.68 sec
SLIM_BPR_Recommender: Epoch 103 of 254. Elapsed time 6.72 sec
SLIM_BPR_Recommender: Epoch 104 of 254. Elapsed time 6.78 sec
SLIM_BPR_Recommender: Epoch 105 of 254. Elapsed time 6.82 sec
SLIM_BPR_Recommender: Epoch 106 of 254. Elapsed time 6.85 sec
SLIM_BPR_Recommender: Epoch 107 of 254. Elapsed time 6.88 sec
SLIM_BPR_Recommender: Epoch 108 of 254. Elapsed time 6.92 sec
SLIM_BPR_Recommender: Epoch 109 of 254. Elapsed time 6.96 sec
SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 7.00 sec
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 7.04 sec
SLIM_BPR_Recommender: Epoch 112 of 254. Elapsed time 7.09 sec
SLIM_BPR_Recommender: Epoch 113 of 254. Elapsed time 7.12 sec
SLIM_BPR_Recommender: Epoch 114 of 254. Elapsed time 7.16 sec
SLIM_BPR_Recommender: Epoch 115 of 254. Elapsed time 7.21 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 232 of 254. Elapsed time 11.72 sec
SLIM_BPR_Recommender: Epoch 233 of 254. Elapsed time 11.76 sec
SLIM_BPR_Recommender: Epoch 234 of 254. Elapsed time 11.80 sec
SLIM_BPR_Recommender: Epoch 235 of 254. Elapsed time 11.84 sec
SLIM_BPR_Recommender: Epoch 236 of 254. Elapsed time 11.88 sec
SLIM_BPR_Recommender: Epoch 237 of 254. Elapsed time 11.92 sec
SLIM_BPR_Recommender: Epoch 238 of 254. Elapsed time 11.95 sec
SLIM_BPR_Recommender: Epoch 239 of 254. Elapsed time 11.99 sec
SLIM_BPR_Recommender: Epoch 240 of 254. Elapsed time 12.03 sec
SLIM_BPR_Recommender: Epoch 241 of 254. Elapsed time 12.08 sec
SLIM_BPR_Recommender: Epoch 242 of 254. Elapsed time 12.11 sec
SLIM_BPR_Recommender: Epoch 243 of 254. Elapsed time 12.15 sec
SLIM_BPR_Recommender: Epoch 244 of 254. Elapsed time 12.19 sec
SLIM_BPR_Recommender: Epoch 245 of 254. Elapsed time 12.23 sec
SLIM_BPR_Recommender: Epoch 246 of 254. Elapsed time 12.26 sec
SLIM_BPR_Recommender: Epoch 247 of 254. Elapsed time 12

MAP 0.0625709736924402 WITH WEIGHTS [0.8, 0.8, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
weights : [0.9, 0.8, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
EvaluatorHoldout: Processed 5000 ( 90.55% ) in 32.76 sec. Users per second: 153
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 36.40 sec. Users per second: 152
MAP 0.06249088234432572 WITH WEIGHTS [0.9, 0.8, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
weights : [1.0, 0.8, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
EvaluatorHoldout: Processed 5000 ( 90.55% ) in 32.78 sec. Users per second: 153
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 36.33 sec. Users per second: 152
MAP 0.06252925233603711 WITH WEIGHTS [1.0, 0.8, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
weights : [1.8, 0.8, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
EvaluatorHoldout: Processed 5000 ( 90.55% ) in 33.60 sec. Users per second: 149
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 37.11 sec. Users per second: 149
MAP 0.06252298172344176 WITH WEIGHTS [1.8, 0.8, 0.00138, 0.8982, 

EvaluatorHoldout: Processed 4000 ( 72.44% ) in 31.04 sec. Users per second: 129
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 43.02 sec. Users per second: 128
MAP 0.062464290921474055 WITH WEIGHTS [0.8, 5, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
weights : [0.8, 10, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 31.01 sec. Users per second: 129
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 42.82 sec. Users per second: 129
MAP 0.06077625114193125 WITH WEIGHTS [0.8, 10, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
weights : [0.8, 50, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 32.17 sec. Users per second: 124
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 45.68 sec. Users per second: 121
MAP 0.058568526633291366 WITH WEIGHTS [0.8, 50, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
weights : [0.8, 80, 0.00138, 0.8982, 0.1, 0.0006, 0.4987505]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 35.74 sec. Users per s

...

EvaluatorHoldout: Processed 4000 ( 72.44% ) in 33.32 sec. Users per second: 120
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 45.25 sec. Users per second: 122
MAP 0.06372306971066694 WITH WEIGHTS [0.8, 2.6, 0.00138, 0.9, 0.1, 0.0006, 0.6599999999999999]
BEST MAP 0.06418784324097589 WITH T 0.5
trials [0.436 0.452 0.468 0.484 0.5   0.52  0.54  0.56  0.58 ]
weights : [0.8, 2.6, 0.00138, 0.43600000000000005, 0.1, 0.0006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 30.94 sec. Users per second: 129
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 42.75 sec. Users per second: 129
MAP 0.06407131697333976 WITH WEIGHTS [0.8, 2.6, 0.00138, 0.43600000000000005, 0.1, 0.0006, 0.6599999999999999]
weights : [0.8, 2.6, 0.00138, 0.452, 0.1, 0.0006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 31.62 sec. Users per second: 126
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 43.22 sec. Users per second: 128
MAP 0.06404618935216773 WITH WEIGHTS [0.8, 2.6, 0.0013

EvaluatorHoldout: Processed 5522 ( 100.00% ) in 50.18 sec. Users per second: 110
MAP 0.06430783140485992 WITH WEIGHTS [0.8, 2.6, 0.00138, 0.58, 0.1, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.00138, 0.58, 0.1, 0.007, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 34.92 sec. Users per second: 115
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 46.71 sec. Users per second: 118
MAP 0.06430424306461821 WITH WEIGHTS [0.8, 2.6, 0.00138, 0.58, 0.1, 0.007, 0.6599999999999999]
weights : [0.8, 2.6, 0.00138, 0.58, 0.1, 0.008, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 30.46 sec. Users per second: 131
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 42.06 sec. Users per second: 131
MAP 0.06430441074406877 WITH WEIGHTS [0.8, 2.6, 0.00138, 0.58, 0.1, 0.008, 0.6599999999999999]
weights : [0.8, 2.6, 0.00138, 0.58, 0.1, 0.009000000000000001, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 31.66 sec. Users per second: 126
EvaluatorHold

EvaluatorHoldout: Processed 5522 ( 100.00% ) in 42.57 sec. Users per second: 130
MAP 0.06379545359298962 WITH WEIGHTS [0.8, 2.6, 0.00138, 0.58, 0.05, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.00138, 0.58, 0.1, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 38.07 sec. Users per second: 105
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 51.99 sec. Users per second: 106
MAP 0.06430783140485992 WITH WEIGHTS [0.8, 2.6, 0.00138, 0.58, 0.1, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.00138, 0.58, 0.5, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 35.85 sec. Users per second: 112
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 48.39 sec. Users per second: 114
MAP 0.06211342095827961 WITH WEIGHTS [0.8, 2.6, 0.00138, 0.58, 0.5, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.00138, 0.58, 1, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 30.77 sec. Users per second: 130
EvaluatorHoldout: Processed 5

EvaluatorHoldout: Processed 4000 ( 72.44% ) in 30.94 sec. Users per second: 129
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 43.25 sec. Users per second: 128
MAP 0.06413262328956694 WITH WEIGHTS [0.8, 2.6, 0.00138, 0.58, 0.16399999999999998, 0.006, 0.6599999999999999]
BEST MAP 0.06430788889495727 WITH T 0.094
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
BEST T:0.094 -> BEST MAP:0.06430788889495727
optimize 2
trials [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 80]
weights : [0.8, 2.6, 0.0001, 0.58, 0.094, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 37.95 sec. Users per second: 105
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 51.79 sec. Users per second: 107
MAP 0.0642203602217

EvaluatorHoldout: Processed 4000 ( 72.44% ) in 31.89 sec. Users per second: 125
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 43.90 sec. Users per second: 126
MAP 0.0643534665650369 WITH WEIGHTS [0.8, 2.6, 0.052000000000000005, 0.58, 0.094, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.054000000000000006, 0.58, 0.094, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 32.77 sec. Users per second: 122
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 44.69 sec. Users per second: 124
MAP 0.06435262477427146 WITH WEIGHTS [0.8, 2.6, 0.054000000000000006, 0.58, 0.094, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05600000000000001, 0.58, 0.094, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.44% ) in 34.96 sec. Users per second: 114
EvaluatorHoldout: Processed 5522 ( 100.00% ) in 48.67 sec. Users per second: 113
MAP 0.0643790821961472 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.58, 0.094, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05

In [11]:
ww_not_std3 = optimize_weights_1at1(URM_train,recs, inits, fits,3,weights = ww_not_std2, normalize_scores = False)


ItemKNNCFRecommender: URM Detected 100 (1.26 %) cold users.
ItemKNNCFRecommender: URM Detected 2719 (10.47 %) cold items.
Similarity column 25975 ( 100 % ), 2592.86 column/sec, elapsed time 0.17 min
ItemKNNCFRecommender: URM Detected 100 (1.26 %) cold users.
ItemKNNCFRecommender: URM Detected 2719 (10.47 %) cold items.
Similarity column 25975 ( 100 % ), 2550.70 column/sec, elapsed time 0.17 min
WeightedHybridScoreRecommender: URM Detected 100 (1.26 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2719 (10.47 %) cold items.
TopPopRecommender: URM Detected 100 (1.26 %) cold users.
TopPopRecommender: URM Detected 2719 (10.47 %) cold items.
P3alphaRecommender: URM Detected 100 (1.26 %) cold users.
P3alphaRecommender: URM Detected 2719 (10.47 %) cold items.
RP3betaRecommender: URM Detected 100 (1.26 %) cold users.
RP3betaRecommender: URM Detected 2719 (10.47 %) cold items.
UserKNNCFRecommender: URM Detected 100 (1.26 %) cold users.
UserKNNCFRecommender: URM Detected 2719 (10.47 %

SLIM_BPR_Recommender: Epoch 100 of 254. Elapsed time 4.04 sec
SLIM_BPR_Recommender: Epoch 101 of 254. Elapsed time 4.07 sec
SLIM_BPR_Recommender: Epoch 102 of 254. Elapsed time 4.09 sec
SLIM_BPR_Recommender: Epoch 103 of 254. Elapsed time 4.12 sec
SLIM_BPR_Recommender: Epoch 104 of 254. Elapsed time 4.15 sec
SLIM_BPR_Recommender: Epoch 105 of 254. Elapsed time 4.17 sec
SLIM_BPR_Recommender: Epoch 106 of 254. Elapsed time 4.20 sec
SLIM_BPR_Recommender: Epoch 107 of 254. Elapsed time 4.23 sec
SLIM_BPR_Recommender: Epoch 108 of 254. Elapsed time 4.25 sec
SLIM_BPR_Recommender: Epoch 109 of 254. Elapsed time 4.28 sec
SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 4.31 sec
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 4.33 sec
SLIM_BPR_Recommender: Epoch 112 of 254. Elapsed time 4.36 sec
SLIM_BPR_Recommender: Epoch 113 of 254. Elapsed time 4.39 sec
SLIM_BPR_Recommender: Epoch 114 of 254. Elapsed time 4.41 sec
SLIM_BPR_Recommender: Epoch 115 of 254. Elapsed time 4.45 sec
SLIM_BPR

EvaluatorHoldout: Processed 4000 ( 72.52% ) in 32.99 sec. Users per second: 121
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 45.08 sec. Users per second: 122
MAP 0.06357254759328579 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.26, 0.094, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05600000000000001, 0.33999999999999997, 0.094, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 32.22 sec. Users per second: 124
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 46.09 sec. Users per second: 120
MAP 0.06378550120965572 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.33999999999999997, 0.094, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05600000000000001, 0.42000000000000004, 0.094, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 37.90 sec. Users per second: 106
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 52.42 sec. Users per second: 105
MAP 0.06368967547806592 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.42000000000000004, 

MAP 0.06404605036727713 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.7, 0.01, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05600000000000001, 0.7, 0.05, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 33.17 sec. Users per second: 121
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 47.43 sec. Users per second: 116
MAP 0.06430365694675792 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.7, 0.05, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05600000000000001, 0.7, 0.1, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 37.61 sec. Users per second: 106
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 51.65 sec. Users per second: 107
MAP 0.0643207499642351 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.7, 0.1, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05600000000000001, 0.7, 0.5, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 33.70 sec. Users per second: 119
EvaluatorHoldout: Processed 5516 ( 100.00

EvaluatorHoldout: Processed 5516 ( 100.00% ) in 52.49 sec. Users per second: 105
MAP 0.06434081884674206 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.7, 0.07200000000000001, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05600000000000001, 0.7, 0.07400000000000001, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 35.65 sec. Users per second: 112
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 47.69 sec. Users per second: 116
MAP 0.06430887713604763 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.7, 0.07400000000000001, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05600000000000001, 0.7, 0.076, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 32.68 sec. Users per second: 122
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 44.46 sec. Users per second: 124
MAP 0.06431982732360525 WITH WEIGHTS [0.8, 2.6, 0.05600000000000001, 0.7, 0.076, 0.006, 0.6599999999999999]
weights : [0.8, 2.6, 0.05600000000000001, 0.7, 0.078, 0.006, 0.659999

...

MAP 0.059011563694045754 WITH WEIGHTS [0.388, 0.0001, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
weights : [0.388, 0.0005, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 31.50 sec. Users per second: 127
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 43.73 sec. Users per second: 126
MAP 0.05901321833221237 WITH WEIGHTS [0.388, 0.0005, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
weights : [0.388, 0.001, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 32.85 sec. Users per second: 122
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 44.88 sec. Users per second: 123
MAP 0.059034476835613726 WITH WEIGHTS [0.388, 0.001, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
weights : [0.388, 0.005, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 32.61 sec. Users per second: 

EvaluatorHoldout: Processed 4000 ( 72.52% ) in 32.07 sec. Users per second: 125
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 44.59 sec. Users per second: 124
MAP 0.06414277832772508 WITH WEIGHTS [0.388, 2.12, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
weights : [0.388, 2.2800000000000002, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 31.55 sec. Users per second: 127
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 43.51 sec. Users per second: 127
MAP 0.06431443715992552 WITH WEIGHTS [0.388, 2.2800000000000002, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
weights : [0.388, 2.44, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6599999999999999]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 34.36 sec. Users per second: 116
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 49.23 sec. Users per second: 112
MAP 0.06440583993178813 WITH WEIGHTS [0.388, 2.44, 0.045200000000000004, 0.7, 0.064, 0.006, 0.6

EvaluatorHoldout: Processed 5516 ( 100.00% ) in 36.75 sec. Users per second: 150
MAP 0.06468240333761403 WITH WEIGHTS [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0018, 0.6599999999999999]
weights : [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0026, 0.6599999999999999]
EvaluatorHoldout: Processed 5000 ( 90.65% ) in 33.18 sec. Users per second: 151
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 37.00 sec. Users per second: 149
MAP 0.06468542485078783 WITH WEIGHTS [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0026, 0.6599999999999999]
weights : [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0034000000000000002, 0.6599999999999999]
EvaluatorHoldout: Processed 5000 ( 90.65% ) in 34.80 sec. Users per second: 144
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 38.92 sec. Users per second: 142
MAP 0.06469181079490038 WITH WEIGHTS [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0034000000000000002, 0.6599999999999999]
weights : [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.00

MAP 0.060272699186233125 WITH WEIGHTS [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.005]
weights : [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.01]
EvaluatorHoldout: Processed 5000 ( 90.65% ) in 35.84 sec. Users per second: 140
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 39.35 sec. Users per second: 140
MAP 0.060257579630232476 WITH WEIGHTS [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.01]
weights : [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.05]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 31.35 sec. Users per second: 128
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 43.10 sec. Users per second: 128
MAP 0.06108002232699535 WITH WEIGHTS [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.05]
weights : [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.1]
EvaluatorHoldout: Processed 4000 ( 72.52% ) in 34.15 sec. Users per second: 117
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 45.04 sec. Users per second: 122
MAP 0.06

EvaluatorHoldout: Processed 5000 ( 90.65% ) in 33.16 sec. Users per second: 151
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 36.73 sec. Users per second: 150
MAP 0.06415876870985812 WITH WEIGHTS [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.74]
weights : [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.76]
EvaluatorHoldout: Processed 5000 ( 90.65% ) in 33.54 sec. Users per second: 149
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 36.96 sec. Users per second: 149
MAP 0.06411079199661215 WITH WEIGHTS [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.76]
weights : [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.78]
EvaluatorHoldout: Processed 5000 ( 90.65% ) in 33.43 sec. Users per second: 150
EvaluatorHoldout: Processed 5516 ( 100.00% ) in 37.04 sec. Users per second: 149
MAP 0.06396343227955371 WITH WEIGHTS [0.388, 2.6, 0.045200000000000004, 0.7, 0.064, 0.0064, 0.78]
BEST MAP 0.06470593996602739 WITH T 0.6599999999999999
--------------------------

In [12]:
ww_std3 = optimize_weights_1at1(URM_train,recs, inits, fits,4,weights = ww_std2, normalize_scores = True)


ItemKNNCFRecommender: URM Detected 99 (1.25 %) cold users.
ItemKNNCFRecommender: URM Detected 2781 (10.71 %) cold items.
Similarity column 25975 ( 100 % ), 3057.64 column/sec, elapsed time 0.14 min
ItemKNNCFRecommender: URM Detected 99 (1.25 %) cold users.
ItemKNNCFRecommender: URM Detected 2781 (10.71 %) cold items.
Similarity column 25975 ( 100 % ), 3150.91 column/sec, elapsed time 0.14 min
WeightedHybridScoreRecommender: URM Detected 99 (1.25 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2781 (10.71 %) cold items.
TopPopRecommender: URM Detected 99 (1.25 %) cold users.
TopPopRecommender: URM Detected 2781 (10.71 %) cold items.
P3alphaRecommender: URM Detected 99 (1.25 %) cold users.
P3alphaRecommender: URM Detected 2781 (10.71 %) cold items.
RP3betaRecommender: URM Detected 99 (1.25 %) cold users.
RP3betaRecommender: URM Detected 2781 (10.71 %) cold items.
UserKNNCFRecommender: URM Detected 99 (1.25 %) cold users.
UserKNNCFRecommender: URM Detected 2781 (10.71 %) cold 

SLIM_BPR_Recommender: Epoch 232 of 254. Elapsed time 8.85 sec
SLIM_BPR_Recommender: Epoch 233 of 254. Elapsed time 8.88 sec
SLIM_BPR_Recommender: Epoch 234 of 254. Elapsed time 8.91 sec
SLIM_BPR_Recommender: Epoch 235 of 254. Elapsed time 8.94 sec
SLIM_BPR_Recommender: Epoch 236 of 254. Elapsed time 8.98 sec
SLIM_BPR_Recommender: Epoch 237 of 254. Elapsed time 9.01 sec
SLIM_BPR_Recommender: Epoch 238 of 254. Elapsed time 9.05 sec
SLIM_BPR_Recommender: Epoch 239 of 254. Elapsed time 9.09 sec
SLIM_BPR_Recommender: Epoch 240 of 254. Elapsed time 9.11 sec
SLIM_BPR_Recommender: Epoch 241 of 254. Elapsed time 9.14 sec
SLIM_BPR_Recommender: Epoch 242 of 254. Elapsed time 9.17 sec
SLIM_BPR_Recommender: Epoch 243 of 254. Elapsed time 9.21 sec
SLIM_BPR_Recommender: Epoch 244 of 254. Elapsed time 9.23 sec
SLIM_BPR_Recommender: Epoch 245 of 254. Elapsed time 9.26 sec
SLIM_BPR_Recommender: Epoch 246 of 254. Elapsed time 9.29 sec
SLIM_BPR_Recommender: Epoch 247 of 254. Elapsed time 9.33 sec
SLIM_BPR

KeyboardInterrupt: 

In [ ]:
ww_not_std4 = optimize_weights_1at1(URM_train,recs, inits, fits,4,weights = ww_not_std3, normalize_scores = False, rinnova_ds = True)


In [ ]:
ww_std4 = optimize_weights_1at1(URM_train,recs, inits, fits,4,weights = ww_std3, normalize_scores = True, , rinnova_ds = True)


# Ottimizzo i risultati dello skopt

{'w0': 4.32825149816237, 'w1': 3.1887159979212667, 'w2': 0.0, 'w3': 6.94062354355586, 'w4': 10.0, 'w5': 10.0, 'w6': 0.0, 'w7': 10.0}

In [37]:
recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender,
    CFW_D_Similarity_Linalg,
    ]
inits = [
        only_URM,only_URM,only_URM,cython,also_ICM,only_URM,CBF,cfw
    ]
fits = [alpha,beta,userknn,cy,iicm,ials,itemcbf,cfw_icm]

skopt_w =[0.20167983999999997, 
          0.22167783999999996, 
          0.01,
          0.7848215199999999, 
          0.30566944, 
          0.6200379999999999, 
          0.85121488, 
          0.28247175999999996]
# [0.432825149816237, 
#            0.31887159979212667, 
#            0.01,
#            0.0, 
#            0.694062354355586, 
#            1, 
#            1, 
#            0.0, 
#            1]
          

In [14]:
ww = optimize_weights_1at1(URM_train,recs, inits, fits,4,weights = skopt_w, normalize_scores = True, rinnova_ds = False)


ItemKNNCFRecommender: URM Detected 92 (1.16 %) cold users.
ItemKNNCFRecommender: URM Detected 2403 (9.25 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 25975 ( 100 % ), 2448.77 column/sec, elapsed time 0.18 min
Similarity column 25975 ( 100 % ), 1441.48 column/sec, elapsed time 0.30 min
WeightedHybridScoreRecommender: URM Detected 92 (1.16 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2403 (9.25 %) cold items.
TopPopRecommender: URM Detected 92 (1.16 %) cold users.
TopPopRecommender: URM Detected 2403 (9.25 %) cold items.
P3alphaRecommender: URM Detected 92 (1.16 %) cold users.
P3alphaRecommender: URM Detected 2403 (9.25 %) cold items.
RP3betaRecommender: URM Detected 92 (1.16 %) cold users.
RP3betaRecommender: URM Detected 2403 (9.25 %) cold items.
UserKNNCFRecommender: URM Detected 92 (1.16 %) cold users.
UserKNNCFRecommender: URM Detected 2403 (9.25 %) cold items.
Ite

SLIM_BPR_Recommender: Epoch 102 of 254. Elapsed time 3.72 sec
SLIM_BPR_Recommender: Epoch 103 of 254. Elapsed time 3.74 sec
SLIM_BPR_Recommender: Epoch 104 of 254. Elapsed time 3.76 sec
SLIM_BPR_Recommender: Epoch 105 of 254. Elapsed time 3.79 sec
SLIM_BPR_Recommender: Epoch 106 of 254. Elapsed time 3.80 sec
SLIM_BPR_Recommender: Epoch 107 of 254. Elapsed time 3.83 sec
SLIM_BPR_Recommender: Epoch 108 of 254. Elapsed time 3.85 sec
SLIM_BPR_Recommender: Epoch 109 of 254. Elapsed time 3.87 sec
SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 3.89 sec
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 3.92 sec
SLIM_BPR_Recommender: Epoch 112 of 254. Elapsed time 3.94 sec
SLIM_BPR_Recommender: Epoch 113 of 254. Elapsed time 3.96 sec
SLIM_BPR_Recommender: Epoch 114 of 254. Elapsed time 3.98 sec
SLIM_BPR_Recommender: Epoch 115 of 254. Elapsed time 4.01 sec
SLIM_BPR_Recommender: Epoch 116 of 254. Elapsed time 4.04 sec
SLIM_BPR_Recommender: Epoch 117 of 254. Elapsed time 4.06 sec
SLIM_BPR

EvaluatorHoldout: Processed 3000 ( 52.28% ) in 35.03 sec. Users per second: 86
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.03 min. Users per second: 93
MAP 0.07068462801222349 WITH WEIGHTS [0.20167983999999997, 0.20007999999999998, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
weights : [0.20167983999999997, 0.30006999999999995, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
EvaluatorHoldout: Processed 4000 ( 69.71% ) in 37.88 sec. Users per second: 106
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 54.47 sec. Users per second: 105
MAP 0.07055459901483774 WITH WEIGHTS [0.20167983999999997, 0.30006999999999995, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
weights : [0.20167983999999997, 0.40005999999999997, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
EvaluatorHoldou

EvaluatorHoldout: Processed 4000 ( 69.71% ) in 38.58 sec. Users per second: 104
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 55.48 sec. Users per second: 103
MAP 0.07058719291681123 WITH WEIGHTS [0.20167983999999997, 0.280072, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
BEST MAP 0.07076196908381345 WITH T 0.22007799999999997
trials [0.2040796 0.2080792 0.2120788 0.2160784 0.220078  0.2240776 0.2280772
 0.2320768 0.2360764]
weights : [0.20167983999999997, 0.20407959999999997, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
EvaluatorHoldout: Processed 4000 ( 69.71% ) in 39.22 sec. Users per second: 102
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 56.08 sec. Users per second: 102
MAP 0.07069105504374638 WITH WEIGHTS [0.20167983999999997, 0.20407959999999997, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
weights : [0.2016798

EvaluatorHoldout: Processed 4000 ( 69.71% ) in 39.43 sec. Users per second: 101
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 55.75 sec. Users per second: 103
MAP 0.0707638449794023 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
weights : [0.20167983999999997, 0.22247775999999997, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
EvaluatorHoldout: Processed 4000 ( 69.71% ) in 39.36 sec. Users per second: 102
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.02 min. Users per second: 94
MAP 0.07075566365412443 WITH WEIGHTS [0.20167983999999997, 0.22247775999999997, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
weights : [0.20167983999999997, 0.22327767999999995, 0.01, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996]
EvaluatorHoldou

...

EvaluatorHoldout: Processed 2000 ( 34.86% ) in 31.79 sec. Users per second: 63
EvaluatorHoldout: Processed 4000 ( 69.71% ) in 1.04 min. Users per second: 64
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.47 min. Users per second: 65
MAP 0.07090485697428849 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.01, 0.5240476, 0.30566943999999996, 0.6200379999999999, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 0.01, 0.5280471999999999, 0.30566943999999996, 0.6200379999999999, 0.85121488, 0.27527248]
EvaluatorHoldout: Processed 3000 ( 52.28% ) in 42.07 sec. Users per second: 71
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.31 min. Users per second: 73
MAP 0.07096293215496939 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.01, 0.5280471999999999, 0.30566943999999996, 0.6200379999999999, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 0.01, 0.5320467999999999, 0.30566943999999996, 0.6200379999999999, 0.8512148

EvaluatorHoldout: Processed 3000 ( 52.28% ) in 36.43 sec. Users per second: 82
EvaluatorHoldout: Processed 5000 ( 87.14% ) in 1.15 min. Users per second: 73
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.32 min. Users per second: 73
MAP 0.06938142950143146 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.01, 0.50964904, 0.30566943999999996, 0.20007999999999998, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 0.01, 0.50964904, 0.30566943999999996, 0.30006999999999995, 0.85121488, 0.27527248]
EvaluatorHoldout: Processed 3000 ( 52.28% ) in 43.11 sec. Users per second: 70
EvaluatorHoldout: Processed 5000 ( 87.14% ) in 1.22 min. Users per second: 68
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.43 min. Users per second: 67
MAP 0.06977086084262643 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.01, 0.50964904, 0.30566943999999996, 0.30006999999999995, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 0.01, 0.50

EvaluatorHoldout: Processed 3000 ( 52.28% ) in 36.65 sec. Users per second: 82
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.17 min. Users per second: 82
MAP 0.07180968260818134 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.01, 0.50964904, 0.30566943999999996, 0.988751125, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 0.01, 0.50964904, 0.30566943999999996, 0.990001, 0.85121488, 0.27527248]
EvaluatorHoldout: Processed 3000 ( 52.28% ) in 38.85 sec. Users per second: 77
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.15 min. Users per second: 83
MAP 0.07180924691630264 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.01, 0.50964904, 0.30566943999999996, 0.990001, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 0.01, 0.50964904, 0.30566943999999996, 0.991250875, 0.85121488, 0.27527248]
EvaluatorHoldout: Processed 3000 ( 52.28% ) in 33.61 sec. Users per second: 89
EvaluatorHoldout: Processed 5738 ( 100.00

EvaluatorHoldout: Processed 4000 ( 69.71% ) in 35.41 sec. Users per second: 113
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.00 min. Users per second: 96
MAP 0.0712477804303498 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.80002, 0.50964904, 0.30566943999999996, 0.993750625, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 0.90001, 0.50964904, 0.30566943999999996, 0.993750625, 0.85121488, 0.27527248]
EvaluatorHoldout: Processed 3000 ( 52.28% ) in 36.93 sec. Users per second: 81
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.14 min. Users per second: 84
MAP 0.07080886254912473 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.90001, 0.50964904, 0.30566943999999996, 0.993750625, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 1.0, 0.50964904, 0.30566943999999996, 0.993750625, 0.85121488, 0.27527248]
EvaluatorHoldout: Processed 3000 ( 52.28% ) in 36.51 sec. Users per second: 82
EvaluatorHoldout: Processed

MAP 0.07215111434921616 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.5360463999999999, 0.50964904, 0.30566943999999996, 0.993750625, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 0.5400459999999999, 0.50964904, 0.30566943999999996, 0.993750625, 0.85121488, 0.27527248]
EvaluatorHoldout: Processed 3000 ( 52.28% ) in 39.43 sec. Users per second: 76
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.30 min. Users per second: 74
MAP 0.07219036811276411 WITH WEIGHTS [0.20167983999999997, 0.22167783999999996, 0.5400459999999999, 0.50964904, 0.30566943999999996, 0.993750625, 0.85121488, 0.27527248]
weights : [0.20167983999999997, 0.22167783999999996, 0.5440455999999999, 0.50964904, 0.30566943999999996, 0.993750625, 0.85121488, 0.27527248]
EvaluatorHoldout: Processed 2000 ( 34.86% ) in 32.68 sec. Users per second: 61
EvaluatorHoldout: Processed 5000 ( 87.14% ) in 1.25 min. Users per second: 67
EvaluatorHoldout: Processed 5738 ( 100.00% ) in 1.41 min. U

In [ ]:
ww2 = optimize_weights_1at1(URM_train,recs, inits, fits,4,weights = ww, normalize_scores = False, rinnova_ds = False)


In [8]:
ww2 = [0.50724928,
 0.31046895999999996,
 0.5344465599999999,
 0.6904309599999999,
 1,
 1,
 0.16968303999999998,
 1]

In [ ]:
ww3 = optimize_weights_1at1(URM_train,recs, inits, fits,4,weights = ww2, normalize_scores = False, rinnova_ds = True)


In [ ]:
ww3 = [0.28807119999999997, 
       0.3072692799999999, 
       0.24327567999999994, 
       0.4520548, 
       1.0, 
       1.0, 
       0.01929808, 
       0.45125487999999997]

In [ ]:
ww4 = optimize_weights_1at1(URM_train,recs, inits, fits,4,weights = ww3, normalize_scores = False, rinnova_ds = True)


In [8]:
#ww4 = [0.08513948799999999, 0.37206279999999997, 0.9640036, 0.33686632, 0.9856134400000001, 1.0, 0.07689232, 1.0]


In [ ]:
ww5 = optimize_weights_1at1(URM_train,recs, inits, fits,4,weights = ww4, normalize_scores = False, rinnova_ds = True)


In [ ]:
ww6 = optimize_weights_1at1CROSS(URM_train,recs, inits, fits,4,weights = ww5, normalize_scores = False)
#FINO A QUI CON KNN

In [13]:
#ww6 = optimize_weights_1at1(URM_train,recs, inits, fits,4,weights = ww5, normalize_scores = False, rinnova_ds = True)


optimize 2
ItemKNNCFRecommender: URM Detected 60 (0.76 %) cold users.
ItemKNNCFRecommender: URM Detected 2344 (9.02 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 25975 ( 100 % ), 4253.00 column/sec, elapsed time 0.10 min
Similarity column 25975 ( 100 % ), 2355.95 column/sec, elapsed time 0.18 min
ItemKNNCFRecommender: URM Detected 60 (0.76 %) cold users.
ItemKNNCFRecommender: URM Detected 2344 (9.02 %) cold items.
Similarity column 25975 ( 100 % ), 4353.78 column/sec, elapsed time 0.10 min
WeightedHybridScoreRecommender: URM Detected 60 (0.76 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2344 (9.02 %) cold items.
TopPopRecommender: URM Detected 60 (0.76 %) cold users.
TopPopRecommender: URM Detected 2344 (9.02 %) cold items.
P3alphaRecommender: URM Detected 60 (0.76 %) cold users.
P3alphaRecommender: URM Detected 2344 (9.02 %) cold items.
RP3betaRecommender: URM Detecte

SLIM_BPR_Recommender: Epoch 100 of 254. Elapsed time 2.42 sec
SLIM_BPR_Recommender: Epoch 101 of 254. Elapsed time 2.44 sec
SLIM_BPR_Recommender: Epoch 102 of 254. Elapsed time 2.47 sec
SLIM_BPR_Recommender: Epoch 103 of 254. Elapsed time 2.49 sec
SLIM_BPR_Recommender: Epoch 104 of 254. Elapsed time 2.51 sec
SLIM_BPR_Recommender: Epoch 105 of 254. Elapsed time 2.54 sec
SLIM_BPR_Recommender: Epoch 106 of 254. Elapsed time 2.56 sec
SLIM_BPR_Recommender: Epoch 107 of 254. Elapsed time 2.58 sec
SLIM_BPR_Recommender: Epoch 108 of 254. Elapsed time 2.61 sec
SLIM_BPR_Recommender: Epoch 109 of 254. Elapsed time 2.63 sec
SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 2.65 sec
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 2.68 sec
SLIM_BPR_Recommender: Epoch 112 of 254. Elapsed time 2.70 sec
SLIM_BPR_Recommender: Epoch 113 of 254. Elapsed time 2.72 sec
SLIM_BPR_Recommender: Epoch 114 of 254. Elapsed time 2.75 sec
SLIM_BPR_Recommender: Epoch 115 of 254. Elapsed time 2.77 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 233 of 254. Elapsed time 5.55 sec
SLIM_BPR_Recommender: Epoch 234 of 254. Elapsed time 5.57 sec
SLIM_BPR_Recommender: Epoch 235 of 254. Elapsed time 5.60 sec
SLIM_BPR_Recommender: Epoch 236 of 254. Elapsed time 5.62 sec
SLIM_BPR_Recommender: Epoch 237 of 254. Elapsed time 5.65 sec
SLIM_BPR_Recommender: Epoch 238 of 254. Elapsed time 5.67 sec
SLIM_BPR_Recommender: Epoch 239 of 254. Elapsed time 5.69 sec
SLIM_BPR_Recommender: Epoch 240 of 254. Elapsed time 5.72 sec
SLIM_BPR_Recommender: Epoch 241 of 254. Elapsed time 5.74 sec
SLIM_BPR_Recommender: Epoch 242 of 254. Elapsed time 5.76 sec
SLIM_BPR_Recommender: Epoch 243 of 254. Elapsed time 5.79 sec
SLIM_BPR_Recommender: Epoch 244 of 254. Elapsed time 5.81 sec
SLIM_BPR_Recommender: Epoch 245 of 254. Elapsed time 5.83 sec
SLIM_BPR_Recommender: Epoch 246 of 254. Elapsed time 5.86 sec
SLIM_BPR_Recommender: Epoch 247 of 254. Elapsed time 5.88 sec
SLIM_BPR_Recommender: Epoch 248 of 254. Elapsed time 5.90 sec
SLIM_BPR

EvaluatorHoldout: Processed 4000 ( 70.67% ) in 32.02 sec. Users per second: 125
EvaluatorHoldout: Processed 5660 ( 100.00% ) in 45.68 sec. Users per second: 124
MAP 0.07246106505519355 WITH WEIGHTS [0.22327767999999995, 0.22007799999999997, 0.50005, 0.32566744, 0.89441056, 0.77682232, 0.09529048, 0.29127087999999995]
weights : [0.22327767999999995, 0.22007799999999997, 0.6000399999999999, 0.32566744, 0.89441056, 0.77682232, 0.09529048, 0.29127087999999995]
EvaluatorHoldout: Processed 4000 ( 70.67% ) in 32.13 sec. Users per second: 125
EvaluatorHoldout: Processed 5660 ( 100.00% ) in 45.45 sec. Users per second: 125
MAP 0.07249495938933084 WITH WEIGHTS [0.22327767999999995, 0.22007799999999997, 0.6000399999999999, 0.32566744, 0.89441056, 0.77682232, 0.09529048, 0.29127087999999995]
weights : [0.22327767999999995, 0.22007799999999997, 0.7000299999999999, 0.32566744, 0.89441056, 0.77682232, 0.09529048, 0.29127087999999995]
EvaluatorHoldout: Processed 4000 ( 70.67% ) in 32.29 sec. Users per

MAP 0.07264064866229167 WITH WEIGHTS [0.22327767999999995, 0.22007799999999997, 0.7720228, 0.32566744, 0.89441056, 0.77682232, 0.09529048, 0.29127087999999995]
weights : [0.22327767999999995, 0.22007799999999997, 0.7760224, 0.32566744, 0.89441056, 0.77682232, 0.09529048, 0.29127087999999995]
EvaluatorHoldout: Processed 4000 ( 70.67% ) in 33.26 sec. Users per second: 120
EvaluatorHoldout: Processed 5660 ( 100.00% ) in 46.37 sec. Users per second: 122
MAP 0.07263829294968861 WITH WEIGHTS [0.22327767999999995, 0.22007799999999997, 0.7760224, 0.32566744, 0.89441056, 0.77682232, 0.09529048, 0.29127087999999995]
weights : [0.22327767999999995, 0.22007799999999997, 0.780022, 0.32566744, 0.89441056, 0.77682232, 0.09529048, 0.29127087999999995]
EvaluatorHoldout: Processed 4000 ( 70.67% ) in 32.18 sec. Users per second: 124
EvaluatorHoldout: Processed 5660 ( 100.00% ) in 46.03 sec. Users per second: 123
MAP 0.07264891468955063 WITH WEIGHTS [0.22327767999999995, 0.22007799999999997, 0.780022, 0.3

...

SLIM_BPR_Recommender: Epoch 57 of 254. Elapsed time 1.35 sec
SLIM_BPR_Recommender: Epoch 58 of 254. Elapsed time 1.37 sec
SLIM_BPR_Recommender: Epoch 59 of 254. Elapsed time 1.39 sec
SLIM_BPR_Recommender: Epoch 60 of 254. Elapsed time 1.42 sec
SLIM_BPR_Recommender: Epoch 61 of 254. Elapsed time 1.44 sec
SLIM_BPR_Recommender: Epoch 62 of 254. Elapsed time 1.46 sec
SLIM_BPR_Recommender: Epoch 63 of 254. Elapsed time 1.49 sec
SLIM_BPR_Recommender: Epoch 64 of 254. Elapsed time 1.51 sec
SLIM_BPR_Recommender: Epoch 65 of 254. Elapsed time 1.53 sec
SLIM_BPR_Recommender: Epoch 66 of 254. Elapsed time 1.56 sec
SLIM_BPR_Recommender: Epoch 67 of 254. Elapsed time 1.58 sec
SLIM_BPR_Recommender: Epoch 68 of 254. Elapsed time 1.60 sec
SLIM_BPR_Recommender: Epoch 69 of 254. Elapsed time 1.63 sec
SLIM_BPR_Recommender: Epoch 70 of 254. Elapsed time 1.65 sec
SLIM_BPR_Recommender: Epoch 71 of 254. Elapsed time 1.67 sec
SLIM_BPR_Recommender: Epoch 72 of 254. Elapsed time 1.70 sec
SLIM_BPR_Recommender: Ep

SLIM_BPR_Recommender: Epoch 190 of 254. Elapsed time 4.52 sec
SLIM_BPR_Recommender: Epoch 191 of 254. Elapsed time 4.54 sec
SLIM_BPR_Recommender: Epoch 192 of 254. Elapsed time 4.57 sec
SLIM_BPR_Recommender: Epoch 193 of 254. Elapsed time 4.59 sec
SLIM_BPR_Recommender: Epoch 194 of 254. Elapsed time 4.61 sec
SLIM_BPR_Recommender: Epoch 195 of 254. Elapsed time 4.64 sec
SLIM_BPR_Recommender: Epoch 196 of 254. Elapsed time 4.66 sec
SLIM_BPR_Recommender: Epoch 197 of 254. Elapsed time 4.68 sec
SLIM_BPR_Recommender: Epoch 198 of 254. Elapsed time 4.71 sec
SLIM_BPR_Recommender: Epoch 199 of 254. Elapsed time 4.73 sec
SLIM_BPR_Recommender: Epoch 200 of 254. Elapsed time 4.76 sec
SLIM_BPR_Recommender: Epoch 201 of 254. Elapsed time 4.78 sec
SLIM_BPR_Recommender: Epoch 202 of 254. Elapsed time 4.80 sec
SLIM_BPR_Recommender: Epoch 203 of 254. Elapsed time 4.82 sec
SLIM_BPR_Recommender: Epoch 204 of 254. Elapsed time 4.85 sec
SLIM_BPR_Recommender: Epoch 205 of 254. Elapsed time 4.87 sec
SLIM_BPR

CFW_D_Similarity_Linalg: Nonzero collaborative cell sum is: 2.78E+03, average is: 2.65E-02, average over all collaborative data is 2.46E-02
Similarity column 25975 ( 100 % ), 3372.77 column/sec, elapsed time 0.13 min
------FITTING END------
trials [1.0000e-04 1.0009e-01 2.0008e-01 3.0007e-01 4.0006e-01 5.0005e-01
 6.0004e-01 7.0003e-01 8.0002e-01 9.0001e-01 1.0000e+00]
weights : [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.0001, 0]
EvaluatorHoldout: Processed 5000 ( 87.49% ) in 33.78 sec. Users per second: 148
EvaluatorHoldout: Processed 5715 ( 100.00% ) in 38.98 sec. Users per second: 147
MAP 0.07497126717736109 WITH WEIGHTS [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.0001, 0]
weights : [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.10009, 0]
EvaluatorHoldout: Processed 

EvaluatorHoldout: Processed 4000 ( 69.99% ) in 32.86 sec. Users per second: 122
EvaluatorHoldout: Processed 5715 ( 100.00% ) in 58.12 sec. Users per second: 98
MAP 0.0750050746247106 WITH WEIGHTS [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.32006799999999996, 0]
weights : [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.340066, 0]
EvaluatorHoldout: Processed 4000 ( 69.99% ) in 35.96 sec. Users per second: 111
EvaluatorHoldout: Processed 5715 ( 100.00% ) in 49.21 sec. Users per second: 116
MAP 0.07500878249456021 WITH WEIGHTS [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.340066, 0]
weights : [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.36006399999999994, 0]
EvaluatorHoldout: Processed 4000 ( 69.99% ) in 3

EvaluatorHoldout: Processed 4000 ( 69.99% ) in 40.73 sec. Users per second: 98
EvaluatorHoldout: Processed 5715 ( 100.00% ) in 1.00 min. Users per second: 95
MAP 0.07506349324830856 WITH WEIGHTS [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.280072, 0]
weights : [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28087192, 0]
EvaluatorHoldout: Processed 4000 ( 69.99% ) in 37.29 sec. Users per second: 107
EvaluatorHoldout: Processed 5715 ( 100.00% ) in 56.19 sec. Users per second: 102
MAP 0.07506349324830856 WITH WEIGHTS [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28087192, 0]
weights : [0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28167184, 0]
EvaluatorHoldout: Processed 3000 ( 52.49% ) in 31.64 sec. Users p

In [14]:
print(ww4)
print(ww6)

[0.22327767999999995, 0.21927807999999996, 0.05049496, 0.32486751999999997, 0.89521048, 0.77682232, 0.10488952, 0.0480952]
[0.20167983999999997, 0.23687631999999997, 0.7848215199999999, 0.30086991999999996, 0.6200379999999999, 0.85121488, 0.28247175999999996, 0]


In [38]:
ww3_notscaled = [0.668922, 0.37206279999999997, 
           0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 1.02,0.484496 ]

In [9]:
from optimize_weights_1at1CROSS import optimize_weights_1at1CROSS

ww_10 = optimize_weights_1at1CROSS(URM_train,recs, inits, fits,2,weights = ww3_notscaled, normalize_scores = False, validations=3)
#FINO A QUI CON KNN
ww_10

optimize 7
TRIALS: [1.0000e-04 3.3430e-02 6.6760e-02 1.0009e-01 1.3342e-01 1.6675e-01
 2.0008e-01 2.3341e-01 2.6674e-01 3.0007e-01 3.3340e-01 3.6673e-01
 4.0006e-01 4.3339e-01 4.6672e-01 5.0005e-01 5.3338e-01 5.6671e-01
 6.0004e-01 6.3337e-01 6.6670e-01 7.0003e-01 7.3336e-01 7.6669e-01
 8.0002e-01 8.3335e-01 8.6668e-01 9.0001e-01 9.3334e-01 9.6667e-01
 1.0000e+00]
ItemKNNCFRecommender: URM Detected 74 (0.93 %) cold users.
ItemKNNCFRecommender: URM Detected 2413 (9.29 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 25975 ( 100 % ), 4586.44 column/sec, elapsed time 0.09 min
Similarity column 25975 ( 100 % ), 1728.88 column/sec, elapsed time 0.25 min
WeightedHybridScoreRecommender: URM Detected 74 (0.93 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2413 (9.29 %) cold items.
TopPopRecommender: URM Detected 74 (0.93 %) cold users.
TopPopRecommender: URM Detected 2413 (9.29 %) 

SLIM_BPR_Recommender: Epoch 96 of 254. Elapsed time 9.02 sec
SLIM_BPR_Recommender: Epoch 97 of 254. Elapsed time 9.09 sec
SLIM_BPR_Recommender: Epoch 98 of 254. Elapsed time 9.17 sec
SLIM_BPR_Recommender: Epoch 99 of 254. Elapsed time 9.42 sec
SLIM_BPR_Recommender: Epoch 100 of 254. Elapsed time 9.62 sec
SLIM_BPR_Recommender: Epoch 101 of 254. Elapsed time 9.74 sec
SLIM_BPR_Recommender: Epoch 102 of 254. Elapsed time 9.84 sec
SLIM_BPR_Recommender: Epoch 103 of 254. Elapsed time 9.92 sec
SLIM_BPR_Recommender: Epoch 104 of 254. Elapsed time 10.05 sec
SLIM_BPR_Recommender: Epoch 105 of 254. Elapsed time 10.14 sec
SLIM_BPR_Recommender: Epoch 106 of 254. Elapsed time 10.27 sec
SLIM_BPR_Recommender: Epoch 107 of 254. Elapsed time 10.33 sec
SLIM_BPR_Recommender: Epoch 108 of 254. Elapsed time 10.46 sec
SLIM_BPR_Recommender: Epoch 109 of 254. Elapsed time 10.60 sec
SLIM_BPR_Recommender: Epoch 110 of 254. Elapsed time 10.80 sec
SLIM_BPR_Recommender: Epoch 111 of 254. Elapsed time 10.90 sec
SLIM

SLIM_BPR_Recommender: Epoch 227 of 254. Elapsed time 20.38 sec
SLIM_BPR_Recommender: Epoch 228 of 254. Elapsed time 20.43 sec
SLIM_BPR_Recommender: Epoch 229 of 254. Elapsed time 20.47 sec
SLIM_BPR_Recommender: Epoch 230 of 254. Elapsed time 20.50 sec
SLIM_BPR_Recommender: Epoch 231 of 254. Elapsed time 20.54 sec
SLIM_BPR_Recommender: Epoch 232 of 254. Elapsed time 20.59 sec
SLIM_BPR_Recommender: Epoch 233 of 254. Elapsed time 20.63 sec
SLIM_BPR_Recommender: Epoch 234 of 254. Elapsed time 20.66 sec
SLIM_BPR_Recommender: Epoch 235 of 254. Elapsed time 20.71 sec
SLIM_BPR_Recommender: Epoch 236 of 254. Elapsed time 20.75 sec
SLIM_BPR_Recommender: Epoch 237 of 254. Elapsed time 20.81 sec
SLIM_BPR_Recommender: Epoch 238 of 254. Elapsed time 20.85 sec
SLIM_BPR_Recommender: Epoch 239 of 254. Elapsed time 20.89 sec
SLIM_BPR_Recommender: Epoch 240 of 254. Elapsed time 20.94 sec
SLIM_BPR_Recommender: Epoch 241 of 254. Elapsed time 20.99 sec
SLIM_BPR_Recommender: Epoch 242 of 254. Elapsed time 21

EvaluatorHoldout: Processed 3000 ( 53.30% ) in 33.03 sec. Users per second: 91
EvaluatorHoldout: Processed 5000 ( 88.84% ) in 1.36 min. Users per second: 61
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 1.67 min. Users per second: 56
MAP 0.06951796717067923 WITH WEIGHTS [0.08513948799999999, 0.37206279999999997, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 0.1689232, 0.06676]
EvaluatorHoldout: Processed 3000 ( 53.30% ) in 40.35 sec. Users per second: 74
EvaluatorHoldout: Processed 5000 ( 88.84% ) in 1.30 min. Users per second: 64
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 1.50 min. Users per second: 63
MAP 0.06968041159381429 WITH WEIGHTS [0.08513948799999999, 0.37206279999999997, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 0.1689232, 0.10009]
EvaluatorHoldout: Processed 3000 ( 53.30% ) in 41.35 sec. Users per second: 73
EvaluatorHoldout: Processed 5000 ( 88.84% ) in 1.30 min. Users per second: 64
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 1.54 min. Users p

EvaluatorHoldout: Processed 3000 ( 53.30% ) in 36.45 sec. Users per second: 82
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 1.10 min. Users per second: 85
MAP 0.06975587469219954 WITH WEIGHTS [0.08513948799999999, 0.37206279999999997, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 0.1689232, 0.90001]
EvaluatorHoldout: Processed 3000 ( 53.30% ) in 34.82 sec. Users per second: 86
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 1.17 min. Users per second: 80
MAP 0.06952986306857875 WITH WEIGHTS [0.08513948799999999, 0.37206279999999997, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 0.1689232, 0.93334]
EvaluatorHoldout: Processed 3000 ( 53.30% ) in 33.71 sec. Users per second: 89
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 1.05 min. Users per second: 89
MAP 0.06970517012018142 WITH WEIGHTS [0.08513948799999999, 0.37206279999999997, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 0.1689232, 0.9666699999999999]
EvaluatorHoldout: Processed 3000 ( 53.30% ) in 33.03 sec

...

EvaluatorHoldout: Processed 5698 ( 100.00% ) in 1.35 min. Users per second: 70
MAP 0.07248546853351272 WITH WEIGHTS [0.6689219999999999, 0.03343, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 1.0222200000000001, 0.484496]
EvaluatorHoldout: Processed 3000 ( 52.65% ) in 43.06 sec. Users per second: 70
EvaluatorHoldout: Processed 5698 ( 100.00% ) in 1.37 min. Users per second: 69
MAP 0.07256527612182372 WITH WEIGHTS [0.6689219999999999, 0.06676, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 1.0222200000000001, 0.484496]
EvaluatorHoldout: Processed 3000 ( 52.65% ) in 43.98 sec. Users per second: 68
EvaluatorHoldout: Processed 5000 ( 87.75% ) in 1.24 min. Users per second: 67
EvaluatorHoldout: Processed 5698 ( 100.00% ) in 1.41 min. Users per second: 67
MAP 0.0725164593595716 WITH WEIGHTS [0.6689219999999999, 0.10009, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 1.0222200000000001, 0.484496]
EvaluatorHoldout: Processed 3000 ( 52.65% ) in 44.40 sec. Users per second: 68

EvaluatorHoldout: Processed 3000 ( 52.65% ) in 39.41 sec. Users per second: 76
EvaluatorHoldout: Processed 5698 ( 100.00% ) in 1.20 min. Users per second: 79
MAP 0.07308893369692691 WITH WEIGHTS [0.6689219999999999, 0.8666799999999999, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 1.0222200000000001, 0.484496]
EvaluatorHoldout: Processed 3000 ( 52.65% ) in 35.97 sec. Users per second: 83
EvaluatorHoldout: Processed 5698 ( 100.00% ) in 1.13 min. Users per second: 84
MAP 0.07327373706200241 WITH WEIGHTS [0.6689219999999999, 0.90001, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 1.0222200000000001, 0.484496]
EvaluatorHoldout: Processed 3000 ( 52.65% ) in 35.52 sec. Users per second: 84
EvaluatorHoldout: Processed 5698 ( 100.00% ) in 1.15 min. Users per second: 82
MAP 0.07312784043651395 WITH WEIGHTS [0.6689219999999999, 0.93334, 0.9680032, 0.33686632, 0.9856134400000001, 0.997778, 1.0222200000000001, 0.484496]
EvaluatorHoldout: Processed 3000 ( 52.65% ) in 35.27 sec. Users p

SLIM_BPR_Recommender: Epoch 70 of 254. Elapsed time 2.33 sec
SLIM_BPR_Recommender: Epoch 71 of 254. Elapsed time 2.35 sec
SLIM_BPR_Recommender: Epoch 72 of 254. Elapsed time 2.38 sec
SLIM_BPR_Recommender: Epoch 73 of 254. Elapsed time 2.41 sec
SLIM_BPR_Recommender: Epoch 74 of 254. Elapsed time 2.43 sec
SLIM_BPR_Recommender: Epoch 75 of 254. Elapsed time 2.46 sec
SLIM_BPR_Recommender: Epoch 76 of 254. Elapsed time 2.49 sec
SLIM_BPR_Recommender: Epoch 77 of 254. Elapsed time 2.51 sec
SLIM_BPR_Recommender: Epoch 78 of 254. Elapsed time 2.54 sec
SLIM_BPR_Recommender: Epoch 79 of 254. Elapsed time 2.56 sec
SLIM_BPR_Recommender: Epoch 80 of 254. Elapsed time 2.59 sec
SLIM_BPR_Recommender: Epoch 81 of 254. Elapsed time 2.62 sec
SLIM_BPR_Recommender: Epoch 82 of 254. Elapsed time 2.65 sec
SLIM_BPR_Recommender: Epoch 83 of 254. Elapsed time 2.67 sec
SLIM_BPR_Recommender: Epoch 84 of 254. Elapsed time 2.70 sec
SLIM_BPR_Recommender: Epoch 85 of 254. Elapsed time 2.73 sec
SLIM_BPR_Recommender: Ep

CFW_D_Similarity_Linalg: Generating train data. Sample 13131506 ( 146.42 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 13699627 ( 152.75 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 14320740 ( 159.67 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 14945409 ( 166.64 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 15571637 ( 173.62 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 16113363 ( 179.66 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 16572008 ( 184.78 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 17154703 ( 191.27 %) 
CFW_D_Similarity_Linalg: Content S structure has 17424123 out of 17714547 ( 98.36%) nonzero collaborative cells
CFW_D_Similarity_Linalg: Nonzero collaborative cell sum is: 4.75E+05, average is: 2.73E-02, average over all collaborative data is 2.37E-02


KeyboardInterrupt: 

In [39]:
from optimize_weights_1at1CROSSparam import optimize_weights_1at1CROSSparam
ww_10 = optimize_weights_1at1CROSSparam(URM_train,recs, inits, fits,1,weights = ww3_notscaled, normalize_scores = False, validations=4,size=80)


optimize 3
TRIALS: [1.0000000e-04 1.2598750e-02 2.5097500e-02 3.7596250e-02 5.0095000e-02
 6.2593750e-02 7.5092500e-02 8.7591250e-02 1.0009000e-01 1.1258875e-01
 1.2508750e-01 1.3758625e-01 1.5008500e-01 1.6258375e-01 1.7508250e-01
 1.8758125e-01 2.0008000e-01 2.1257875e-01 2.2507750e-01 2.3757625e-01
 2.5007500e-01 2.6257375e-01 2.7507250e-01 2.8757125e-01 3.0007000e-01
 3.1256875e-01 3.2506750e-01 3.3756625e-01 3.5006500e-01 3.6256375e-01
 3.7506250e-01 3.8756125e-01 4.0006000e-01 4.1255875e-01 4.2505750e-01
 4.3755625e-01 4.5005500e-01 4.6255375e-01 4.7505250e-01 4.8755125e-01
 5.0005000e-01 5.1254875e-01 5.2504750e-01 5.3754625e-01 5.5004500e-01
 5.6254375e-01 5.7504250e-01 5.8754125e-01 6.0004000e-01 6.1253875e-01
 6.2503750e-01 6.3753625e-01 6.5003500e-01 6.6253375e-01 6.7503250e-01
 6.8753125e-01 7.0003000e-01 7.1252875e-01 7.2502750e-01 7.3752625e-01
 7.5002500e-01 7.6252375e-01 7.7502250e-01 7.8752125e-01 8.0002000e-01
 8.1251875e-01 8.2501750e-01 8.3751625e-01 8.5001500e-01 8

SLIM_BPR_Recommender: Epoch 82 of 254. Elapsed time 2.75 sec
SLIM_BPR_Recommender: Epoch 83 of 254. Elapsed time 2.78 sec
SLIM_BPR_Recommender: Epoch 84 of 254. Elapsed time 2.81 sec
SLIM_BPR_Recommender: Epoch 85 of 254. Elapsed time 2.84 sec
SLIM_BPR_Recommender: Epoch 86 of 254. Elapsed time 2.87 sec
SLIM_BPR_Recommender: Epoch 87 of 254. Elapsed time 2.89 sec
SLIM_BPR_Recommender: Epoch 88 of 254. Elapsed time 2.92 sec
SLIM_BPR_Recommender: Epoch 89 of 254. Elapsed time 2.94 sec
SLIM_BPR_Recommender: Epoch 90 of 254. Elapsed time 2.98 sec
SLIM_BPR_Recommender: Epoch 91 of 254. Elapsed time 3.03 sec
SLIM_BPR_Recommender: Epoch 92 of 254. Elapsed time 3.06 sec
SLIM_BPR_Recommender: Epoch 93 of 254. Elapsed time 3.09 sec
SLIM_BPR_Recommender: Epoch 94 of 254. Elapsed time 3.11 sec
SLIM_BPR_Recommender: Epoch 95 of 254. Elapsed time 3.14 sec
SLIM_BPR_Recommender: Epoch 96 of 254. Elapsed time 3.17 sec
SLIM_BPR_Recommender: Epoch 97 of 254. Elapsed time 3.19 sec
SLIM_BPR_Recommender: Ep

SLIM_BPR_Recommender: Epoch 215 of 254. Elapsed time 7.75 sec
SLIM_BPR_Recommender: Epoch 216 of 254. Elapsed time 7.77 sec
SLIM_BPR_Recommender: Epoch 217 of 254. Elapsed time 7.80 sec
SLIM_BPR_Recommender: Epoch 218 of 254. Elapsed time 7.82 sec
SLIM_BPR_Recommender: Epoch 219 of 254. Elapsed time 7.85 sec
SLIM_BPR_Recommender: Epoch 220 of 254. Elapsed time 7.88 sec
SLIM_BPR_Recommender: Epoch 221 of 254. Elapsed time 7.90 sec
SLIM_BPR_Recommender: Epoch 222 of 254. Elapsed time 7.93 sec
SLIM_BPR_Recommender: Epoch 223 of 254. Elapsed time 7.96 sec
SLIM_BPR_Recommender: Epoch 224 of 254. Elapsed time 7.99 sec
SLIM_BPR_Recommender: Epoch 225 of 254. Elapsed time 8.03 sec
SLIM_BPR_Recommender: Epoch 226 of 254. Elapsed time 8.06 sec
SLIM_BPR_Recommender: Epoch 227 of 254. Elapsed time 8.11 sec
SLIM_BPR_Recommender: Epoch 228 of 254. Elapsed time 8.14 sec
SLIM_BPR_Recommender: Epoch 229 of 254. Elapsed time 8.16 sec
SLIM_BPR_Recommender: Epoch 230 of 254. Elapsed time 8.19 sec
SLIM_BPR

KeyboardInterrupt: 

In [ ]:
ww_10

# SUBMISSION

In [27]:
only_URM = {
        "URM_train" : URM_all
    }
userknn = {}
userknn["topK"] = 121
userknn["shrink"] = 2

cython = {
        "URM_train" : URM_all,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_all,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_all,
        "ICM_train" : ICM_all
    }

ikn = ItemKNNCFRecommender(URM_all)
ikn.fit(**itemknn)

im = ItemIcmKNNCFRecommender(URM_train, ICM_all)
im.fit(**iicm)

cfw1  =  {
        "URM_train" : URM_all,
        "ICM" : ICM_all,
        "S_matrix_target": im.W_sparse
    }

cfw2  =  {
        "URM_train" : URM_all,
        "ICM" : ICM_all,
        "S_matrix_target": ikn.W_sparse
    }

recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender,
    CFW_D_Similarity_Linalg,
    ]
inits = [
        only_URM,only_URM,only_URM,cython,also_ICM,only_URM,CBF,cfw1
    ]
fits = [alpha,beta,userknn,cy,iicm,ials,itemcbf,cfw_icm]


sub_w = ww3_notscaled
# [0.08513948799999999, 0.37206279999999997, 0.9680032, 0.33686632, 
#            0.9856134400000001, 1.0, 0.07689232, 1.0]


ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
Similarity column 25975 ( 100 % ), 2244.19 column/sec, elapsed time 0.19 min
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 25975 ( 100 % ), 2678.94 column/sec, elapsed time 0.16 min
Similarity column 25975 ( 100 % ), 1531.87 column/sec, elapsed time 0.28 min


In [28]:
from WeightedHybridV10 import WeightedHybridScoreRecommender

recommender = WeightedHybridScoreRecommender(URM_all, recs, inits, False)
recommender.fit(fits,ww3_notscaled)


WeightedHybridScoreRecommender: URM Detected 1079 (4.15 %) cold items.
TopPopRecommender: URM Detected 1079 (4.15 %) cold items.
P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.
RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
UserKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
CFW_D_Similarity_Linalg: URM Detected 1079 (4.15 %) cold items.
--------FITTING START-------
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: RP3betaRecommender-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7519.37 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: SLIM_BPR_Recommender-------
Una

SLIM_BPR_Recommender: Epoch 118 of 254. Elapsed time 6.03 sec
SLIM_BPR_Recommender: Epoch 119 of 254. Elapsed time 6.07 sec
SLIM_BPR_Recommender: Epoch 120 of 254. Elapsed time 6.11 sec
SLIM_BPR_Recommender: Epoch 121 of 254. Elapsed time 6.15 sec
SLIM_BPR_Recommender: Epoch 122 of 254. Elapsed time 6.19 sec
SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 6.23 sec
SLIM_BPR_Recommender: Epoch 124 of 254. Elapsed time 6.27 sec
SLIM_BPR_Recommender: Epoch 125 of 254. Elapsed time 6.30 sec
SLIM_BPR_Recommender: Epoch 126 of 254. Elapsed time 6.34 sec
SLIM_BPR_Recommender: Epoch 127 of 254. Elapsed time 6.37 sec
SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 6.41 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 6.44 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 6.47 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 6.50 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 6.53 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 6.57 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 250 of 254. Elapsed time 13.92 sec
SLIM_BPR_Recommender: Epoch 251 of 254. Elapsed time 14.12 sec
SLIM_BPR_Recommender: Epoch 252 of 254. Elapsed time 14.68 sec
SLIM_BPR_Recommender: Epoch 253 of 254. Elapsed time 15.62 sec
SLIM_BPR_Recommender: Epoch 254 of 254. Elapsed time 17.16 sec
SLIM_BPR_Recommender: Terminating at epoch 254. Elapsed time 1.86 min
Deallocating Cython objects
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 2348.80 column/sec, elapsed time 0.18 min
Similarity column 25975 ( 100 % ), 1317.37 column/sec, elapsed time 0.33 min
--------FITTING IN PROGRESS: IALSRecommender-------
IALSRecommender: Epoch 1 of 45. Elapsed time 11.84 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 24.33 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 35.74 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 48.79 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 1.02 min
IALSRecommender: Epoch 6 of 45. Elapsed time 1.21 

In [29]:
recommender.setWeights(ww3_notscaled)

In [30]:
import time
URM_file = open(r"datasets/data_target_users_test.csv", 'r')

URM_file.seek(0)
URM_tuples = []
URM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in URM_file:
    URM_tuples.append(int(line.replace("\n","")))

usersToPred = np.array(URM_tuples)

start_time = time.time()#BUONE METRICHE PER IL TEMPO QUI
import csv

with open('provadisperata.csv', mode='w',newline='') as recFile:
    
    recFile = csv.writer(recFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    recFile.writerow(['user_id', 'item_list'])
    recs=recommender.recommend(usersToPred, cutoff=10)
    for user_id,rec in zip(usersToPred,recs):
        rec_string =' '.join(map(str, rec[:10]))
        recFile.writerow([user_id, rec_string])
    
end_time = time.time()
print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")

Ha finito in questo tempo: 112.36144804954529 secondi


## CHECK performance
## L'ultimo qui e' quello con map mega alto su kaggle

In [5]:
#fa 3 train/test split e poi fa la media dei map per fare un po di "cross validation" e vedere su 3 quale metodo va meglio
URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

URM_train2, URM_test2 = train_test_holdout(URM_all, train_perc = 0.8)
evaluator_test2 = EvaluatorHoldout(URM_test2, cutoff_list=[10])

URM_train3, URM_test3 = train_test_holdout(URM_all, train_perc = 0.8)
evaluator_test3 = EvaluatorHoldout(URM_test3, cutoff_list=[10])

In [7]:
#devo rifittare con nuovo train/test split


userknn = {}
userknn["topK"] = 102
userknn["shrink"] = 1

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 254
cy["topK"] = 881
cy["positive_threshold_BPR"] = 0.9765
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565

hybridv2={}
#bayesianSearchhybridV2
#recommenders usati: [alpha,beta,userknn,uicm,cy,itemknn,iicm]
hybridv2["w0"]=0.001
hybridv2["w1"]=0.5593506236737317
hybridv2["w2"]=0.23267769028384777
hybridv2["w3"]=0.10077656126622785
hybridv2["w4"]=0.0001
hybridv2["w5"]=0.0001
hybridv2["w6"]=0.8397993296794579

itemcbf={}
itemcbf["topK"]=107
itemcbf["shrink"]=18.384308506219117
itemcbf["normalize"] = True
itemcbf["feature_weighting"] = 'BM25'
itemcbf["similarity"] = 'jaccard'

recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    #UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender
    ]



only_URM = {
        "URM_train" : URM_train
    }

cython = {
        "URM_train" : URM_train,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_train,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_train,
        "ICM_train" : ICM_all
    }

inits = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]

In [8]:
only_URM = {
        "URM_train" : URM_train2
    }

cython = {
        "URM_train" : URM_train2,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_train2,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_train2,
        "ICM_train" : ICM_all
    }
inits2 = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]

In [9]:
only_URM = {
        "URM_train" : URM_train3
    }

cython = {
        "URM_train" : URM_train3,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_train3,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_train3,
        "ICM_train" : ICM_all
    }
inits3 = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]

In [10]:
fits = [alpha,beta,cy,iicm,ials,itemcbf]

In [11]:
recsToFit = []
for rec, init in zip(recs, inits):
    recsToFit.append(rec(**init))
for rec, fit in zip(recsToFit, fits):
    rec.fit(**fit)

P3alphaRecommender: URM Detected 73 (0.92 %) cold users.
P3alphaRecommender: URM Detected 2431 (9.36 %) cold items.
RP3betaRecommender: URM Detected 73 (0.92 %) cold users.
RP3betaRecommender: URM Detected 2431 (9.36 %) cold items.
ItemKNNCFRecommender: URM Detected 73 (0.92 %) cold users.
ItemKNNCFRecommender: URM Detected 2431 (9.36 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 73 (0.92 %) cold users.
IALSRecommender: URM Detected 2431 (9.36 %) cold items.
ItemKNNCBFRecommender: URM Detected 73 (0.92 %) cold users.
ItemKNNCBFRecommender: URM Detected 2431 (9.36 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.03 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 0.06 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 0.11 sec
SLIM_BPR_Recommender: Epoch 4 of 254. Elapsed time 0.15 sec
SLIM_

SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 3.16 sec
SLIM_BPR_Recommender: Epoch 124 of 254. Elapsed time 3.18 sec
SLIM_BPR_Recommender: Epoch 125 of 254. Elapsed time 3.20 sec
SLIM_BPR_Recommender: Epoch 126 of 254. Elapsed time 3.22 sec
SLIM_BPR_Recommender: Epoch 127 of 254. Elapsed time 3.24 sec
SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 3.25 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 3.27 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 3.29 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 3.31 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 3.33 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 3.35 sec
SLIM_BPR_Recommender: Epoch 134 of 254. Elapsed time 3.37 sec
SLIM_BPR_Recommender: Epoch 135 of 254. Elapsed time 3.39 sec
SLIM_BPR_Recommender: Epoch 136 of 254. Elapsed time 3.41 sec
SLIM_BPR_Recommender: Epoch 137 of 254. Elapsed time 3.43 sec
SLIM_BPR_Recommender: Epoch 138 of 254. Elapsed time 3.44 sec
SLIM_BPR

Deallocating Cython objects
Similarity column 25975 ( 100 % ), 4875.77 column/sec, elapsed time 0.09 min
Similarity column 25975 ( 100 % ), 2719.01 column/sec, elapsed time 0.16 min
IALSRecommender: Epoch 1 of 45. Elapsed time 5.72 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 10.19 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 14.60 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 19.75 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 23.91 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 28.45 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 33.72 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 38.02 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 42.18 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 47.33 sec
IALSRecommender: Epoch 11 of 45. Elapsed time 52.10 sec
IALSRecommender: Epoch 12 of 45. Elapsed time 56.66 sec
IALSRecommender: Epoch 13 of 45. Elapsed time 1.03 min
IALSRecommender: Epoch 14 of 45. Elapsed time 1.11 min
IALSRecommender: Epoch 15 of 45. Elapsed time 

In [12]:
recsToFit2 = []
for rec, init in zip(recs, inits2):
    recsToFit2.append(rec(**init))
for rec, fit in zip(recsToFit2, fits):
    rec.fit(**fit)
    
#passo URM_train2 e recstoFit2

P3alphaRecommender: URM Detected 58 (0.73 %) cold users.
P3alphaRecommender: URM Detected 2418 (9.31 %) cold items.
RP3betaRecommender: URM Detected 58 (0.73 %) cold users.
RP3betaRecommender: URM Detected 2418 (9.31 %) cold items.
ItemKNNCFRecommender: URM Detected 58 (0.73 %) cold users.
ItemKNNCFRecommender: URM Detected 2418 (9.31 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 58 (0.73 %) cold users.
IALSRecommender: URM Detected 2418 (9.31 %) cold items.
ItemKNNCBFRecommender: URM Detected 58 (0.73 %) cold users.
ItemKNNCBFRecommender: URM Detected 2418 (9.31 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.02 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 0.06 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 0.10 sec
SLIM_BPR_Recommender: Epoch 4 of 254. Elapsed time 0.14 sec
SLIM_

SLIM_BPR_Recommender: Terminating at epoch 254. Elapsed time 30.47 sec
Deallocating Cython objects
Similarity column 25975 ( 100 % ), 4735.01 column/sec, elapsed time 0.09 min
Similarity column 25975 ( 100 % ), 2614.63 column/sec, elapsed time 0.17 min
IALSRecommender: Epoch 1 of 45. Elapsed time 4.88 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 9.78 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 13.95 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 18.50 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 23.63 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 27.81 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 32.17 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 37.30 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 41.75 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 46.34 sec
IALSRecommender: Epoch 11 of 45. Elapsed time 50.99 sec
IALSRecommender: Epoch 12 of 45. Elapsed time 55.58 sec
IALSRecommender: Epoch 13 of 45. Elapsed time 1.01 min
IALSRecommender: Epoch 14 of 45

In [13]:
recsToFit3 = []
for rec, init in zip(recs, inits3):
    recsToFit3.append(rec(**init))
for rec, fit in zip(recsToFit3, fits):
    rec.fit(**fit)
    
#passo URM_train2 e recstoFit2

P3alphaRecommender: URM Detected 67 (0.84 %) cold users.
P3alphaRecommender: URM Detected 2429 (9.35 %) cold items.
RP3betaRecommender: URM Detected 67 (0.84 %) cold users.
RP3betaRecommender: URM Detected 2429 (9.35 %) cold items.
ItemKNNCFRecommender: URM Detected 67 (0.84 %) cold users.
ItemKNNCFRecommender: URM Detected 2429 (9.35 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 67 (0.84 %) cold users.
IALSRecommender: URM Detected 2429 (9.35 %) cold items.
ItemKNNCBFRecommender: URM Detected 67 (0.84 %) cold users.
ItemKNNCBFRecommender: URM Detected 2429 (9.35 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.02 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 0.04 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 0.08 sec
SLIM_BPR_Recommender: Epoch 4 of 254. Elapsed time 0.12 sec
SLIM_

SLIM_BPR_Recommender: Epoch 123 of 254. Elapsed time 3.05 sec
SLIM_BPR_Recommender: Epoch 124 of 254. Elapsed time 3.08 sec
SLIM_BPR_Recommender: Epoch 125 of 254. Elapsed time 3.10 sec
SLIM_BPR_Recommender: Epoch 126 of 254. Elapsed time 3.12 sec
SLIM_BPR_Recommender: Epoch 127 of 254. Elapsed time 3.14 sec
SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 3.17 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 3.19 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 3.21 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 3.23 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 3.25 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 3.27 sec
SLIM_BPR_Recommender: Epoch 134 of 254. Elapsed time 3.29 sec
SLIM_BPR_Recommender: Epoch 135 of 254. Elapsed time 3.31 sec
SLIM_BPR_Recommender: Epoch 136 of 254. Elapsed time 3.33 sec
SLIM_BPR_Recommender: Epoch 137 of 254. Elapsed time 3.36 sec
SLIM_BPR_Recommender: Epoch 138 of 254. Elapsed time 3.38 sec
SLIM_BPR

Deallocating Cython objects
Similarity column 25975 ( 100 % ), 4746.94 column/sec, elapsed time 0.09 min
Similarity column 25975 ( 100 % ), 2720.09 column/sec, elapsed time 0.16 min
IALSRecommender: Epoch 1 of 45. Elapsed time 4.43 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 8.53 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 12.59 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 17.32 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 21.40 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 25.41 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 29.89 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 34.49 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 38.66 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 42.98 sec
IALSRecommender: Epoch 11 of 45. Elapsed time 47.53 sec
IALSRecommender: Epoch 12 of 45. Elapsed time 52.16 sec
IALSRecommender: Epoch 13 of 45. Elapsed time 56.50 sec
IALSRecommender: Epoch 14 of 45. Elapsed time 1.01 min
IALSRecommender: Epoch 15 of 45. Elapsed time 

In [14]:
#check num 1
#Config 622
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFit)
best_parameters={'w0': 0.7771253987559757, 'w1': 2.283910304115099, 'w2': 10.0, 'w3': 1.8063059091826554, 'w4': 6.234284335332363, 'w5': 6.4128417124725345}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFit2)
best_parameters={'w0': 0.7771253987559757, 'w1': 2.283910304115099, 'w2': 10.0, 'w3': 1.8063059091826554, 'w4': 6.234284335332363, 'w5': 6.4128417124725345}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFit3)
best_parameters={'w0': 0.7771253987559757, 'w1': 2.283910304115099, 'w2': 10.0, 'w3': 1.8063059091826554, 'w4': 6.234284335332363, 'w5': 6.4128417124725345}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2431 (9.36 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 58 (0.73 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2418 (9.31 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 67 (0.84 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2429 (9.35 %) cold items.
--------FITTING...-------


In [16]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
#print("MAP  recommeder test 1: "+str(MAP)+"\n")

result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

EvaluatorHoldout: Processed 5656 ( 100.00% ) in 15.74 sec. Users per second: 359
MAP  recommeder test 1: 0.07399989928131133

EvaluatorHoldout: Processed 5662 ( 100.00% ) in 15.48 sec. Users per second: 366
MAP  recommeder test 2: 0.07816959888669146

EvaluatorHoldout: Processed 5680 ( 100.00% ) in 16.01 sec. Users per second: 355
MAP  recommeder test 2: 0.07586226632834922


AVERAGE: [0.07601058816545067]


In [17]:
#check num 2 0.073 on validation
#config 1252
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFit)
best_parameters={'w0': 0.9291195521014475, 'w1': 2.9323133970481736, 'w2': 9.20584043071056, 'w3': 2.398420310624778, 'w4': 7.371572182380706, 'w5': 7.741397895020033}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFit2)
best_parameters={'w0': 0.9291195521014475, 'w1': 2.9323133970481736, 'w2': 9.20584043071056, 'w3': 2.398420310624778, 'w4': 7.371572182380706, 'w5': 7.741397895020033}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFit3)
best_parameters={'w0': 0.9291195521014475, 'w1': 2.9323133970481736, 'w2': 9.20584043071056, 'w3': 2.398420310624778, 'w4': 7.371572182380706, 'w5': 7.741397895020033}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2431 (9.36 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 58 (0.73 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2418 (9.31 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 67 (0.84 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2429 (9.35 %) cold items.
--------FITTING...-------


In [18]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
#print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP2)+"\n")


result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

EvaluatorHoldout: Processed 5656 ( 100.00% ) in 15.65 sec. Users per second: 361
MAP  recommeder test 1: 0.07417012870454105

EvaluatorHoldout: Processed 5662 ( 100.00% ) in 15.58 sec. Users per second: 363
MAP  recommeder test 2: 0.07850470843157566

EvaluatorHoldout: Processed 5680 ( 100.00% ) in 16.76 sec. Users per second: 339
MAP  recommeder test 2: 0.07601762146401331


AVERAGE: [0.07623081953337667]


In [19]:
#check num 3, 0.073 on validation
#config 1257 should be best ever
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFit)
best_parameters={'w0': 0.7814026405172689, 'w1': 2.9739152141760856, 'w2': 9.282586619873573, 'w3': 2.4611257593408284, 'w4': 7.584599871341396, 'w5': 7.67927578225758}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFit2)
best_parameters={'w0': 0.7814026405172689, 'w1': 2.9739152141760856, 'w2': 9.282586619873573, 'w3': 2.4611257593408284, 'w4': 7.584599871341396, 'w5': 7.67927578225758}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFit3)
best_parameters={'w0': 0.7814026405172689, 'w1': 2.9739152141760856, 'w2': 9.282586619873573, 'w3': 2.4611257593408284, 'w4': 7.584599871341396, 'w5': 7.67927578225758}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2431 (9.36 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 58 (0.73 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2418 (9.31 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 67 (0.84 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2429 (9.35 %) cold items.
--------FITTING...-------


In [20]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
#print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

EvaluatorHoldout: Processed 5656 ( 100.00% ) in 14.92 sec. Users per second: 379
MAP  recommeder test 1: 0.07401475921351033

EvaluatorHoldout: Processed 5662 ( 100.00% ) in 15.98 sec. Users per second: 354
MAP  recommeder test 2: 0.07840869448731903

EvaluatorHoldout: Processed 5680 ( 100.00% ) in 15.41 sec. Users per second: 369
MAP  recommeder test 2: 0.07594194418605982


AVERAGE: [0.07612179929562973]


In [21]:

#--------!!!!! QUESTO E' QUELLO CON MAP 0.091 SU KAGGLE !!!!!!!!!!!!!!!!--------


#check num 4 config 725
#validation MAP: 0.0702271
#test MAP: 0.0649211
recommender=WeightedHybridV3V4forBayesianSearch(URM_train,ICM_all,recsToFit)
best_parameters={'w0': 0.6464719543114522, 'w1': 3.0330662257244967, 'w2': 10.0, 'w3': 2.5929646423942043, 'w4': 7.558045116812191, 'w5': 7.4137901570719755}
recommender.fit(**best_parameters)

recommender2=WeightedHybridV3V4forBayesianSearch(URM_train2,ICM_all,recsToFit2)
best_parameters={'w0': 0.6464719543114522, 'w1': 3.0330662257244967, 'w2': 10.0, 'w3': 2.5929646423942043, 'w4': 7.558045116812191, 'w5': 7.4137901570719755}
recommender2.fit(**best_parameters)

recommender3=WeightedHybridV3V4forBayesianSearch(URM_train3,ICM_all,recsToFit3)
best_parameters={'w0': 0.6464719543114522, 'w1': 3.0330662257244967, 'w2': 10.0, 'w3': 2.5929646423942043, 'w4': 7.558045116812191, 'w5': 7.4137901570719755}
recommender3.fit(**best_parameters)

WeightedHybridV3V4forBayesianSearch: URM Detected 73 (0.92 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2431 (9.36 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 58 (0.73 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2418 (9.31 %) cold items.
--------FITTING...-------
WeightedHybridV3V4forBayesianSearch: URM Detected 67 (0.84 %) cold users.
WeightedHybridV3V4forBayesianSearch: URM Detected 2429 (9.35 %) cold items.
--------FITTING...-------


In [22]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict[10]["MAP"]
#print("MAP  recommeder test 1: "+str(MAP)+"\n")
result_dict, _ = evaluator_test2.evaluateRecommender(recommender2)
MAP2 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP2)+"\n")

result_dict, _ = evaluator_test3.evaluateRecommender(recommender3)
MAP3 = result_dict[10]["MAP"]
#print("MAP  recommeder test 2: "+str(MAP3)+"\n")

mappozzo=(MAP+MAP2+MAP3)/3
print("\nAVERAGE: ["+str(mappozzo)+"]")

EvaluatorHoldout: Processed 5656 ( 100.00% ) in 14.88 sec. Users per second: 380
MAP  recommeder test 1: 0.07438775925039447

EvaluatorHoldout: Processed 5662 ( 100.00% ) in 14.87 sec. Users per second: 381
MAP  recommeder test 2: 0.07820063724169618

EvaluatorHoldout: Processed 5680 ( 100.00% ) in 15.59 sec. Users per second: 364
MAP  recommeder test 2: 0.07605121425200215


AVERAGE: [0.07621320358136427]


## rifitto tutto ecc x kaggle

In [23]:
#devo rifittare con nuovo train/test split

userknn = {}
userknn["topK"] = 102
userknn["shrink"] = 1

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 254
cy["topK"] = 881
cy["positive_threshold_BPR"] = 0.9765
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.547615508822564
alpha["topK"] = 500

beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565

hybridv2={}
#bayesianSearchhybridV2
#recommenders usati: [alpha,beta,userknn,uicm,cy,itemknn,iicm]
hybridv2["w0"]=0.001
hybridv2["w1"]=0.5593506236737317
hybridv2["w2"]=0.23267769028384777
hybridv2["w3"]=0.10077656126622785
hybridv2["w4"]=0.0001
hybridv2["w5"]=0.0001
hybridv2["w6"]=0.8397993296794579

itemcbf={}
itemcbf["topK"]=107
itemcbf["shrink"]=18.384308506219117
itemcbf["normalize"] = True
itemcbf["feature_weighting"] = 'BM25'
itemcbf["similarity"] = 'jaccard'

# 9 recomm
fits = [alpha,beta,cy,iicm,ials,itemcbf]

recs = [
    P3alphaRecommender,
    RP3betaRecommender,
    #UserKNNCFRecommender,
    #UserIcmKNNCFRecommender,
    SLIM_BPR_Cython,
    #ItemKNNCFRecommender,
    ItemIcmKNNCFRecommender,
    IALSRecommender,
    ItemKNNCBFRecommender
    ]




only_URM = {
        "URM_train" : URM_all
    }

cython = {
        "URM_train" : URM_all,
        "recompile_cython" : False,
        "verbose" : False
    }

also_ICM = {
        "URM_train" : URM_all,
        "ICM" : ICM_all
    }

CBF = {
        "URM_train" : URM_all,
        "ICM_train" : ICM_all
    }

inits = [
        only_URM,only_URM,cython,also_ICM,only_URM,CBF 
    ]

In [ ]:
recsToFit = []
for rec, init in zip(recs, inits):
    recsToFit.append(rec(**init))
for rec, fit in zip(recsToFit, fits):
    rec.fit(**fit)

P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.
RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
IALSRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 254. Elapsed time 0.03 sec
SLIM_BPR_Recommender: Epoch 2 of 254. Elapsed time 0.11 sec
SLIM_BPR_Recommender: Epoch 3 of 254. Elapsed time 0.17 sec
SLIM_BPR_Recommender: Epoch 4 of 254. Elapsed time 0.22 sec
SLIM_BPR_Recommender: Epoch 5 of 254. Elapsed time 0.25 sec
SLIM_BPR_Recommender: Epoch 6 of 254. Elapsed time 0.29 sec
SLIM_BPR_Recommender: Epoch 7 of 254. Elapsed time 0.32 sec
SLIM_BPR_Recommender: Epoch 8 of 254. Elapsed time 0.35 sec
SLIM_BPR_Recommender: Epoch 9 of 254. Elapsed time 0

SLIM_BPR_Recommender: Epoch 128 of 254. Elapsed time 3.50 sec
SLIM_BPR_Recommender: Epoch 129 of 254. Elapsed time 3.52 sec
SLIM_BPR_Recommender: Epoch 130 of 254. Elapsed time 3.54 sec
SLIM_BPR_Recommender: Epoch 131 of 254. Elapsed time 3.56 sec
SLIM_BPR_Recommender: Epoch 132 of 254. Elapsed time 3.58 sec
SLIM_BPR_Recommender: Epoch 133 of 254. Elapsed time 3.60 sec
SLIM_BPR_Recommender: Epoch 134 of 254. Elapsed time 3.63 sec
SLIM_BPR_Recommender: Epoch 135 of 254. Elapsed time 3.65 sec
SLIM_BPR_Recommender: Epoch 136 of 254. Elapsed time 3.67 sec
SLIM_BPR_Recommender: Epoch 137 of 254. Elapsed time 3.70 sec
SLIM_BPR_Recommender: Epoch 138 of 254. Elapsed time 3.72 sec
SLIM_BPR_Recommender: Epoch 139 of 254. Elapsed time 3.74 sec
SLIM_BPR_Recommender: Epoch 140 of 254. Elapsed time 3.76 sec
SLIM_BPR_Recommender: Epoch 141 of 254. Elapsed time 3.79 sec
SLIM_BPR_Recommender: Epoch 142 of 254. Elapsed time 3.81 sec
SLIM_BPR_Recommender: Epoch 143 of 254. Elapsed time 3.83 sec
SLIM_BPR

IALSRecommender: Epoch 3 of 45. Elapsed time 12.95 sec
IALSRecommender: Epoch 4 of 45. Elapsed time 18.45 sec
IALSRecommender: Epoch 5 of 45. Elapsed time 25.33 sec
IALSRecommender: Epoch 6 of 45. Elapsed time 29.76 sec
IALSRecommender: Epoch 7 of 45. Elapsed time 35.72 sec
IALSRecommender: Epoch 8 of 45. Elapsed time 41.59 sec
IALSRecommender: Epoch 9 of 45. Elapsed time 46.64 sec
IALSRecommender: Epoch 10 of 45. Elapsed time 51.59 sec
IALSRecommender: Epoch 11 of 45. Elapsed time 55.85 sec
IALSRecommender: Epoch 12 of 45. Elapsed time 1.00 min
IALSRecommender: Epoch 13 of 45. Elapsed time 1.08 min
IALSRecommender: Epoch 14 of 45. Elapsed time 1.16 min
IALSRecommender: Epoch 15 of 45. Elapsed time 1.24 min
IALSRecommender: Epoch 16 of 45. Elapsed time 1.31 min
IALSRecommender: Epoch 17 of 45. Elapsed time 1.39 min
IALSRecommender: Epoch 18 of 45. Elapsed time 1.46 min
IALSRecommender: Epoch 19 of 45. Elapsed time 1.53 min
IALSRecommender: Epoch 20 of 45. Elapsed time 1.61 min
IALSReco

In [ ]:
recommender=WeightedHybridV3V4forBayesianSearch(URM_all,ICM_all,recsToFit)
best_parameters={}
recommender.fit(**best_parameters)

In [ ]:
import time
URM_file = open(r"data_target_users_test.csv", 'r')

URM_file.seek(0)
URM_tuples = []
URM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in URM_file:
    URM_tuples.append(int(line.replace("\n","")))

usersToPred = np.array(URM_tuples)

start_time = time.time()#BUONE METRICHE PER IL TEMPO QUI
import csv

with open('OUTPUTS\outputIbridoSuperTuning9algos.csv', mode='w',newline='') as recFile:
    
    recFile = csv.writer(recFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    recFile.writerow(['user_id', 'item_list'])
    recs=recommender.recommend(usersToPred, cutoff=10)
    for user_id,rec in zip(usersToPred,recs):
        rec_string =' '.join(map(str, rec[:10]))
        recFile.writerow([user_id, rec_string])
    
end_time = time.time()
print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")